 언어 모델이 해결할 수 있는 것
=> 지시만으로 어려운 문제들을 해결 가능
=> 번역, 요약, 말투 변경, 작문 등…

 언어 모델의 한계
=> 학습 지식을 벗어난 정보에 대해서는 답변 불가능

 언어 모델의 한계를 벗어나는 방법론들의 등장
=> RAG(Retrieval-Augemnted Generation), ReACT(Reasoning and Acting)

이러한 방법론들을 적용하면서 언어 모델 애플리케이션 개발: Langchain


* Model I/O: 프롬프트 준비, 언어 모델 호출, 결과 수신
* Retrieval: 외부 지식을 LLM에 주입. ChatPDF, CSV 파일 기반 답변
* Memory: 과거의 대화를 장/단기로 기억. 이전 문맥을 고려한 답변.
* Chains: 여러 모듈을 통합하는 기능. 단독 사용 용도 X
* Agents: ReACT나 Function Calling 기법을 사용해 외부와 상호 작용.
* Callbacks: 다양한 이벤트 발생을 처리 가능. 단독 사용 용도 X


In [ ]:
# tiktoken은 Embedding 실습을 위해 필요
!pip install openai langchain tiktoken langchain_community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.18
    Uninstalling langchain-0.3.18:
      Successfully uninstalled langchain-0.3.18


아래 링크에서 OpenAI API Key를 발급받으세요.  
링크: https://platform.openai.com/

In [ ]:
import os
os.environ['OPENAI_API_KEY'] =  "여러분의 Key 값"

In [ ]:
import openai

openai.__version__

'1.57.4'

# 1. Langchain 시작하기

## 1-1. ChatOpenAI

OpenAI 사의 채팅 전용 Large Language Model(llm) 입니다.

객체를 생성할 때 다음을 옵션 값을 지정할 수 있습니다. 옵션에 대한 상세 설명은 다음과 같습니다.

`temperature`
- 사용할 샘플링 온도는 0과 2 사이에서 선택합니다. 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.

`max_tokens`
- 채팅 완성에서 생성할 토큰의 최대 개수입니다.

`model_name`: 적용 가능한 모델 리스트

- 링크: https://platform.openai.com/docs/models

In [ ]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4o-mini",  # 모델명
)

# 질의내용
question = "세종대왕이 누구인지 설명해주세요"

# 질의
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='세종대왕(1397-1450)은 조선의 제4대 왕으로, 본명은 이도(李祹)입니다. 그는 1418년에 왕위에 올라 1450년까지 통치하였으며, 그의 통치 기간은 조선 역사에서 가장 빛나는 시기로 평가받고 있습니다.\n\n세종대왕은 특히 한글, 즉 훈민정음을 창제한 왕으로 유명합니다. 한글은 한국어를 표기하기 위해 만든 문자로, 일반 국민이 쉽게 읽고 쓸 수 있도록 고안되었습니다. 이는 당시의 한자 중심의 문자 체계에서 벗어나, 모든 계층의 사람들이 자신의 언어를 사용할 수 있는 기회를 제공한 혁신적인 업적입니다.\n\n그 외에도 세종대왕은 과학, 농업, 음악, 천문학 등 다양한 분야에서 많은 발전을 이끌었습니다. 예를 들어, 그는 측우기(비를 측정하는 기구)와 같은 기구를 개발하고, 농업 기술을 개선하기 위한 여러 정책을 시행했습니다. 또한, 음악과 관련하여 정간보와 같은 악보 체계를 정립하기도 했습니다.\n\n세종대왕은 그의 통치 아래에서 문화와 과학이 크게 발전하였고, 백성을 위한 정책을 많이 시행하여 국민의 삶의 질을 향상시켰습니다. 그의 업적은 오늘날까지도 많은 사람들에게 기억되고 있으며, 한국 역사에서 가장 존경받는 왕 중 한 명으로 여겨집니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 326, 'prompt_tokens': 16, 'total_tokens': 342, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 's

## 1-2. 프롬프트 템플릿

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
- `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 {}는 변수를 나타냅니다.
- `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- `input_variables`는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.
- 사용법: 리스트 형식으로 변수 이름을 정의합니다.

In [ ]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{who}가 누구인지 설명해주세요"

# 템플릿 완성
prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['who'], input_types={}, partial_variables={}, template='{who}가 누구인지 설명해주세요')

프롬프트 템플릿은 {who}는 빈 칸. 즉, 변할 수 있는 값으로 두고 일종의 템플릿을 만들어두는 것입니다.

.format()을 통해 명시적으로 추가하여 프롬프트 완성본을 미리 볼 수 있습니다.

In [ ]:
prompt.format(who="오바마")

'오바마가 누구인지 설명해주세요'

## 1-3. LLMChain 객체

`LLMChain`

- LLMChain은 특정 PromptTemplate와 연결된 체인 객체를 생성합니다  

사용법
- `prompt`: 앞서 정의한 PromptTemplate 객체를 사용합니다.
- `llm`: 언어 모델을 나타냅니다.

```
코드 예시

from langchain.chains import LLMChain

# 연결된 체인(Chain)객체 생성
llm_chain = prompt | llm
```

LLMChain을 사용하기 위해서는 앞서 1.1. ChatOpenAI에서 본 바와 같이 llm이 이미 선언되어져 있어야 합니다.

In [ ]:
from langchain_openai import ChatOpenAI

# llm 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4o-mini",  # 모델명
)

# 연결된 체인(Chain)객체 생성
llm_chain = prompt | llm

정리해보면 현재 llm과 프롬프트 템플릿이 chain을 통해 연결된 상태입니다. 다만, 아직 `{who}`는 현재 변수. 즉, 빈 값 상태로 있습니다.  

이 빈 값을 채울 때는 invoke() 안에 {"변수명": "변수의 값"}으로 전달하면 됩니다.

사용 가능한 변수는 `who`밖에 없는데, 임의로 다른 변수에 대해서 값을 넣는 것을 시도해봅시다.

In [ ]:
llm_chain.invoke({"location": "서울"})

KeyError: "Input to PromptTemplate is missing variables {'who'}.  Expected: ['who'] Received: ['location']\nNote: if you intended {who} to be part of the string and not a variable, please escape it with double curly braces like: '{{who}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

Missing some input keys: {'who'}라는 에러가 발생하는데 who에 대한 값을 채우라는 에러입니다.

이제 who에 직접적으로 값을 넣어 정말로 호출해보겠습니다.

In [ ]:
llm_chain.invoke({"who": "세종대왕"})

AIMessage(content='세종대왕(1397-1450)은 조선의 제4대 왕으로, 본명은 이도(李祹)입니다. 그는 1418년에 왕위에 올라 1450년까지 통치하였으며, 조선 역사에서 가장 위대한 왕 중 한 사람으로 평가받고 있습니다.\n\n세종대왕은 한글, 즉 훈민정음을 창제한 것으로 가장 잘 알려져 있습니다. 한글은 한국어를 표기하기 위해 만든 문자로, 일반 국민들이 쉽게 읽고 쓸 수 있도록 고안되었습니다. 이는 당시 유교적 엘리트 중심의 사회에서 문맹률을 낮추고, 국민의 교육 수준을 높이는 데 큰 기여를 하였습니다.\n\n그 외에도 세종대왕은 과학, 농업, 음악, 천문학 등 다양한 분야에서 많은 업적을 남겼습니다. 예를 들어, 그는 측우기(비를 측정하는 기구)와 같은 기계를 발명하고, 농업 기술을 발전시켜 농민들의 삶을 개선하려고 노력했습니다. 또한, 천문학적 관측을 통해 별자리와 달력을 정리하는 등의 작업도 수행했습니다.\n\n세종대왕은 인재를 중시하고, 백성을 사랑하는 통치자로서, 그의 통치 기간은 조선의 문화와 과학이 발전한 황금기로 여겨집니다. 그의 업적은 오늘날에도 많은 사람들에게 기억되고 있으며, 한국의 역사와 문화에 큰 영향을 미쳤습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 326, 'prompt_tokens': 16, 'total_tokens': 342, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fi

In [ ]:
llm_chain.invoke({"who": "이순신 장군"})

AIMessage(content="이순신(李舜臣, 1545-1598)은 조선 중기의 군인으로, 임진왜란(1592-1598) 동안 일본의 침략에 맞서 싸운 대표적인 장군입니다. 그는 특히 해군의 지휘관으로서 뛰어난 전략과 전술로 유명하며, 그의 전투 중 가장 잘 알려진 것은 명량 해전과 한산도 해전입니다.\n\n이순신은 조선의 해군을 강화하고, 철갑선인 판옥선을 개발하여 해상 전투에서의 우위를 점했습니다. 그의 전투에서의 승리는 조선의 국방에 큰 기여를 하였고, 그는 조선의 역사에서 가장 존경받는 인물 중 하나로 평가받고 있습니다.\n\n그의 업적은 단순히 군사적 승리에 그치지 않고, 충성과 희생정신, 그리고 국가에 대한 헌신으로도 많은 사람들에게 귀감이 되고 있습니다. 이순신은 또한 '난중일기'라는 일기를 남겨, 전쟁의 상황과 자신의 생각을 기록하여 후세에 중요한 역사적 자료로 남겼습니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 249, 'prompt_tokens': 17, 'total_tokens': 266, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'stop', 'logprobs': None}, id='run-74e20ce0-1023-44e3-a084-4671b813966b-0', usage_metadata={'input_tokens': 

과금 구조: https://openai.com/pricing

### 1-3-1. 2개 이상의 변수를 템플릿 안에 정의

변수가 2개 이상이라는 것 외에는 위에서 실습한 것과 사용 방법은 동일

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
# 질문 템플릿 형식 정의
template = "{time1}에서 가장 용맹한 위인과 {time2}에서 가장 용맹한 위인은 누구인지 궁금해"

# 템플릿 완성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['time1', 'time2'], input_types={}, partial_variables={}, template='{time1}에서 가장 용맹한 위인과 {time2}에서 가장 용맹한 위인은 누구인지 궁금해')

In [ ]:
# llm 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)

In [ ]:
# 연결된 체인(Chain)객체 생성
llm_chain = prompt | llm

In [ ]:
llm_chain

PromptTemplate(input_variables=['time1', 'time2'], input_types={}, partial_variables={}, template='{time1}에서 가장 용맹한 위인과 {time2}에서 가장 용맹한 위인은 누구인지 궁금해')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7c6944305630>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7c692b921ed0>, root_client=<openai.OpenAI object at 0x7c692b8c0bb0>, root_async_client=<openai.AsyncOpenAI object at 0x7c69443059c0>, temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=2048)

In [ ]:
print(llm_chain.invoke({"time1": "고려", "time2": "조선"}))

content='고려에서 가장 용맹한 위인으로는 김유신이 유명합니다. 그는 고려의 장군으로서 많은 전투에서 승리를 거두었고, 고려의 안정과 번영을 위해 헌신한 위인으로 평가받고 있습니다.\n\n조선에서 가장 용맹한 위인으로는 이순신 장군이 유명합니다. 그는 조선의 해상 전투에서 맹활약하여 일본의 침략을 막는 데 큰 공헌을 하였으며, 조선의 해군을 발전시키고 국가를 지키는 데 헌신한 위인으로 평가받고 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 212, 'prompt_tokens': 47, 'total_tokens': 259, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-eec911b4-14db-4d25-8d13-205b7c4201de-0' usage_metadata={'input_tokens': 47, 'output_tokens': 212, 'total_tokens': 259, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### 1-3-2. Stream

다음과 같이 streaming=True 로 설정하고 스트리밍으로 답변을 받기 위한 StreamingStdOutCallbackHandler() 을 콜백으로 지정합니다.

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [ ]:
# 질의내용
question = "세종대왕의 이름을 알려줘"

# 스트리밍으로 답변 출력
response = llm.invoke(question)

세종대왕의 본명은 이도입니다. 그러나 그는 세종대왕으로 더 잘 알려져 있습니다.

## 1-4. 프롬프트 템플릿 심화

### 1-4-1. 함수를 프롬프트로 전달

시간이나 날짜에 따라서 프롬프트가 매일 변경되어야 한다면 함수 자체를 프롬프트와 연결할 수 있습니다.

In [ ]:
from datetime import datetime

# 월 일 형식으로 오늘 날짜를 반환하는 함수
def get_today():
    now = datetime.now()
    return now.strftime("%B %d")

get_today()

'December 14'

In [ ]:
prompt_template = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요.",
    input_variables=["n"],
    partial_variables={"today": get_today},  # partial_variables에 함수를 전달
)

In [ ]:
prompt_template

PromptTemplate(input_variables=['n'], input_types={}, partial_variables={'today': <function get_today at 0x7c692b8b96c0>}, template='오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요.')

위에서 프롬프트에서 변수는 {today}와 {n}이지만 partial_variable을 보면 'today'의 값으로는 함수가 전달된 상태로 n의 값은 전달되지 않은 상태이다. .format()을 통해서 n의 값을 마저 전달하면 최종 프롬프트가 완성된다.

In [ ]:
prompt_template.format(n=5)

'오늘의 날짜는 December 14 입니다. 오늘이 생일인 유명인 5명을 나열해 주세요.'

# 2. 유틸리티

## 2-1. 캐싱

https://python.langchain.com/docs/modules/model_io/llms/llm_caching

LangChain은 LLM을 위한 선택적 캐싱 레이어를 제공합니다.

이는 두 가지 이유로 유용합니다:

- 동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 API 호출 횟수를 줄여 비용을 절감 할 수 있습니다.
- LLM 제공업체에 대한 API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수 있습니다.  

모델과 프롬프트를 생성합니다

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

### 캐시 사용을 위한 코드 추가 ###
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
### 캐시 사용을 위한 코드 추가 ###

# llm 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)

# 프롬프트 생성
prompt = PromptTemplate.from_template("{who} 에 대해서 짧게 설명해줘")
prompt

PromptTemplate(input_variables=['who'], input_types={}, partial_variables={}, template='{who} 에 대해서 짧게 설명해줘')

In [ ]:
# 연결된 체인(Chain)객체 생성
llm_chain = prompt | llm

코드 앞에 %%time을 함께 사용해서 실행하면 소요 시간을 확인할 수 있습니다.

In [ ]:
%%time
llm_chain.invoke({"who": "세종대왕"})

CPU times: user 21.7 ms, sys: 3.08 ms, total: 24.8 ms
Wall time: 2.2 s


AIMessage(content='세종대왕은 조선시대 4대 왕 중 하나로, 조선시대를 대표하는 왕으로 꼽힙니다. 세종대왕은 한글을 창제하고 학문과 문화를 발전시키는 데 큰 공헌을 한 위인으로 알려져 있습니다. 또한 세종대왕은 국방력을 강화하고 행정체제를 정비하여 조선시대의 번영을 이룩하는 데 기여한 위인으로 평가받고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 161, 'prompt_tokens': 25, 'total_tokens': 186, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-41757f57-0cec-413d-94f2-f89338938bf9-0', usage_metadata={'input_tokens': 25, 'output_tokens': 161, 'total_tokens': 186, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

현재 캐시가 동작 중이므로 동일한 호출을 다시 해보겠습니다. 캐시가 동작하여 동일한 질문에 대해서는 동일한 답변을 빠르게 얻을 수 있습니다.

In [ ]:
%%time
llm_chain.invoke({"who": "세종대왕"})

CPU times: user 987 µs, sys: 0 ns, total: 987 µs
Wall time: 966 µs


AIMessage(content='세종대왕은 조선시대 4대 왕 중 하나로, 조선시대를 대표하는 왕으로 꼽힙니다. 세종대왕은 한글을 창제하고 학문과 문화를 발전시키는 데 큰 공헌을 한 위인으로 알려져 있습니다. 또한 세종대왕은 국방력을 강화하고 행정체제를 정비하여 조선시대의 번영을 이룩하는 데 기여한 위인으로 평가받고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 161, 'prompt_tokens': 25, 'total_tokens': 186, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-41757f57-0cec-413d-94f2-f89338938bf9-0', usage_metadata={'input_tokens': 25, 'output_tokens': 161, 'total_tokens': 186, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 2-2. 토큰 사용량 확인하기

https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

get_openai_callback을 이용하여 토큰 사용량을 추적할 수 있습니다.

In [ ]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [ ]:
with get_openai_callback() as cb:
    result = llm.invoke("안녕 너는 누구야")
    print(cb)

Tokens Used: 82
	Prompt Tokens: 19
	Completion Tokens: 63
Successful Requests: 1
Total Cost (USD): $0.00010400000000000001


In [ ]:
result

AIMessage(content='안녕, 나는 인공지능 기반 챗봇이야. 너의 질문에 답변해주는데 도움을 줄 수 있어. 무엇을 도와줄까?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 19, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b32b7329-6076-4ae0-9fac-6b406df7de02-0', usage_metadata={'input_tokens': 19, 'output_tokens': 63, 'total_tokens': 82, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# 3. 입력 프롬프트 확인하기

In [ ]:
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [ ]:
# llm 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)

# 프롬프트 생성
prompt = PromptTemplate.from_template("{who} 에 대해서 짧게 설명해줘")
prompt

PromptTemplate(input_variables=['who'], input_types={}, partial_variables={}, template='{who} 에 대해서 짧게 설명해줘')

In [ ]:
# 연결된 체인(Chain)객체 생성
llm_chain = prompt | llm

In [ ]:
# invoke()를 할 때 , config={'callbacks': [ConsoleCallbackHandler()]}를 함께 전달하면 입력 프롬프트까지 출력
result = llm_chain.invoke({"who":"이순신 장군"}, config={'callbacks': [ConsoleCallbackHandler()]})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "who": "이순신 장군"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "who": "이순신 장군"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 이순신 장군 에 대해서 짧게 설명해줘"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.94s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "이순신 장군은 조선시대의 무신이자 민족영웅으로, 조선의 해군을 이끌었던 뛰어난 무장력을 가진 장수입니다. 일본의 침략을 막기 위해 전쟁을 치르고, 명량 해전에서 일본 함대를 물리치는 등 업적을 남겼습니다. 그의 뛰어난 전략과 무기력으로 많은 사람들에게 존경받고 있습니다.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain"

In [ ]:
print(result)

content='이순신 장군은 조선시대의 무신이자 민족영웅으로, 조선의 해군을 이끌었던 뛰어난 무장력을 가진 장수입니다. 일본의 침략을 막기 위해 전쟁을 치르고, 명량 해전에서 일본 함대를 물리치는 등 업적을 남겼습니다. 그의 뛰어난 전략과 무기력으로 많은 사람들에게 존경받고 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 27, 'total_tokens': 184, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-9d058b88-cdbb-4b70-849f-f365393627c9-0' usage_metadata={'input_tokens': 27, 'output_tokens': 157, 'total_tokens': 184, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
print(result.content)

이순신 장군은 조선시대의 무신이자 민족영웅으로, 조선의 해군을 이끌었던 뛰어난 무장력을 가진 장수입니다. 일본의 침략을 막기 위해 전쟁을 치르고, 명량 해전에서 일본 함대를 물리치는 등 업적을 남겼습니다. 그의 뛰어난 전략과 무기력으로 많은 사람들에게 존경받고 있습니다.


# 3. 메모리

ChatGPT와 대화하면 현재의 대화를 이어나갈 때 과거의 대화를 기반으로 대화하는 것을 느낄 수 있습니다.

대부분의 LLM(Large Language Models) 애플리케이션은 대화형 인터페이스를 가지고 있습니다. 대화의 중요한 구성 요소 중 하나는 이전 대화에 있는 정보를 참조할 수 있는 능력입니다.

이전 대화에서 주요한 정보를 기억할 수 있는 이 능력을 우리는 "메모리(memory)"라고 부릅니다. LangChain은 시스템에 메모리를 추가하기 위한 다양한 유틸리티를 제공합니다. 이러한 유틸리티는 단독으로 사용될 수 있으며 체인(chain)에 원활하게 통합될 수 있습니다.

## 3-1. RunnableWithMessageHistory

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# llm 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4o",  # 모델명
)

# 질문 템플릿 형식 정의
template= """당신은 위키북스에서 만든 친절한 위키봇입니다. 누가 당신에게 누구냐고 묻거든 위키북스에서 만든 위키봇이라고 소개하세요.
그 외의 답변은 최선을 다해서 친절하게 답변하세요.

Current Conversation: {history}

Human: {input}

AI:"""

# 템플릿 완성
prompt = PromptTemplate(
        template=template, input_variables=['history', 'input']
    )
print(prompt)

input_variables=['history', 'input'] input_types={} partial_variables={} template='당신은 위키북스에서 만든 친절한 위키봇입니다. 누가 당신에게 누구냐고 묻거든 위키북스에서 만든 위키봇이라고 소개하세요.\n그 외의 답변은 최선을 다해서 친절하게 답변하세요.\n\nCurrent Conversation: {history}\n\nHuman: {input}\n\nAI:'


In [ ]:
# Chat History를 저장할 store 딕셔너리를 생성
store = {}

In [ ]:
# 연결된 체인(Chain)객체 생성
llm_chain = prompt | llm

In [ ]:
# 세션 ID를 생성. 여기서는 임의로 "test"로 지정합니다.
session_id = "test"

# 만약 현재 생성한 세션 ID가 기존에 존재하지 않는다면 store에 추가
if session_id not in store:
    store[session_id] = ChatMessageHistory()

In [ ]:
print(store)

{'test': InMemoryChatMessageHistory(messages=[])}


store 딕셔너리에 "test"를 key로 하고 InMemoryChatMessageHistory()를 value로 하는 원소가 추가된 것을 확인할 수 있다.

In [ ]:
session_history = store[session_id]  # 안전하게 세션 기록을 변수에 할당

store[session_id]를 이번에는 session_history에 저장해서 출력한다.

In [ ]:
session_history

InMemoryChatMessageHistory(messages=[])

In [ ]:
with_message_history = RunnableWithMessageHistory(
    llm_chain,
    lambda session_id: session_history,  # 직접 참조를 반환하도록 수정
    input_messages_key="input",
    history_messages_key="history"
)

이때 "test"라는 세션 내에서 과거 대화가 관리되므로 "test"라고 호출한 대화 한정으로는 과거 대화를 관리할 수 있다.  
invoke() 시에 config={"configurable": {"session_id": "test"}를 같이 넣어서 호출한다.

In [ ]:
# 주어진 메시지와 설정으로 체인을 실행합니다.
result = with_message_history.invoke(
    {"input": "당신은 누구입니까?"},
    config={"configurable": {"session_id": "test"}},
)

print(result)

content='저는 위키북스에서 만든 친절한 위키봇입니다. 무엇을 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 81, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9faba9f038', 'finish_reason': 'stop', 'logprobs': None} id='run-b87fae0f-f84e-4548-820a-4415f8a1bf4e-0' usage_metadata={'input_tokens': 81, 'output_tokens': 25, 'total_tokens': 106, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
print(result.content)

저는 위키북스에서 만든 친절한 위키봇입니다. 무엇을 도와드릴까요?


Warning은 신경쓰지 않아도 됩니다. (https://github.com/langchain-ai/langchain/issues/24713)  

출력 결과를 보면 당신은 누구냐는 질문에 '위키북스에서 만든 위키봇'이라는 답변을 한다.  
session_history를 출력해보면 현재의 질답이 저장되어져 있는 것을 볼 수 있다.

In [ ]:
session_history

InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 위키북스에서 만든 친절한 위키봇입니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 81, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9faba9f038', 'finish_reason': 'stop', 'logprobs': None}, id='run-b87fae0f-f84e-4548-820a-4415f8a1bf4e-0', usage_metadata={'input_tokens': 81, 'output_tokens': 25, 'total_tokens': 106, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})])

store dictionary에도 'test'를 key 값으로 히스토리가 저장되어져 있다.

In [ ]:
print(store)

{'test': InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 위키북스에서 만든 친절한 위키봇입니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 81, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9faba9f038', 'finish_reason': 'stop', 'logprobs': None}, id='run-b87fae0f-f84e-4548-820a-4415f8a1bf4e-0', usage_metadata={'input_tokens': 81, 'output_tokens': 25, 'total_tokens': 106, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})])}


In [ ]:
result = with_message_history.invoke(
    {"input": "'홍길동'으로 삼행시 해줘"},
    config={"configurable": {"session_id": "test"}},
)

print(result)

content="물론이죠! '홍길동'으로 삼행시를 지어볼게요.\n\n홍: 홍익인간의 정신을 품고,  \n길: 길을 잃지 않고 바르게 나아가며,  \n동: 동서남북 어디서든 빛나는 사람이 되자.  \n\n어떠신가요? 더 도와드릴 것이 있으면 말씀해 주세요!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 363, 'total_tokens': 448, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9faba9f038', 'finish_reason': 'stop', 'logprobs': None} id='run-b37e88c4-bc7d-4fd7-b8c7-df71813366ed-0' usage_metadata={'input_tokens': 363, 'output_tokens': 85, 'total_tokens': 448, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
print(result.content)

물론이죠! '홍길동'으로 삼행시를 지어볼게요.

홍: 홍익인간의 정신을 품고,  
길: 길을 잃지 않고 바르게 나아가며,  
동: 동서남북 어디서든 빛나는 사람이 되자.  

어떠신가요? 더 도와드릴 것이 있으면 말씀해 주세요!


In [ ]:
result = with_message_history.invoke(
    {"input": "좀 감성적인 느낌으로 해줘"},
    config={"configurable": {"session_id": "test"}},
)

print(result)

content="물론이죠! 감성적인 느낌으로 '홍길동' 삼행시를 지어볼게요.\n\n홍: 홍조 띤 하늘 아래,  \n길: 길게 드리운 그림자 속에서,  \n동: 동화 같은 꿈을 꾸는 우리.  \n\n어떠신가요? 더 도와드릴 것이 있으면 말씀해 주세요!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 708, 'total_tokens': 787, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9faba9f038', 'finish_reason': 'stop', 'logprobs': None} id='run-430fa74f-7e29-4fc9-81a5-fe92c2a45bcd-0' usage_metadata={'input_tokens': 708, 'output_tokens': 79, 'total_tokens': 787, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
print(result.content)

물론이죠! 감성적인 느낌으로 '홍길동' 삼행시를 지어볼게요.

홍: 홍조 띤 하늘 아래,  
길: 길게 드리운 그림자 속에서,  
동: 동화 같은 꿈을 꾸는 우리.  

어떠신가요? 더 도와드릴 것이 있으면 말씀해 주세요!


In [ ]:
print("메시지 히스토리:")
for message in session_history.messages:
    print(f"{message.__class__.__name__}: {message.content}")
    print('--' * 50)

메시지 히스토리:
HumanMessage: 당신은 누구입니까?
----------------------------------------------------------------------------------------------------
AIMessage: 저는 위키북스에서 만든 친절한 위키봇입니다. 무엇을 도와드릴까요?
----------------------------------------------------------------------------------------------------
HumanMessage: '홍길동'으로 삼행시 해줘
----------------------------------------------------------------------------------------------------
AIMessage: 물론이죠! '홍길동'으로 삼행시를 지어볼게요.

홍: 홍익인간의 정신을 품고,  
길: 길을 잃지 않고 바르게 나아가며,  
동: 동서남북 어디서든 빛나는 사람이 되자.  

어떠신가요? 더 도와드릴 것이 있으면 말씀해 주세요!
----------------------------------------------------------------------------------------------------
HumanMessage: 좀 감성적인 느낌으로 해줘
----------------------------------------------------------------------------------------------------
AIMessage: 물론이죠! 감성적인 느낌으로 '홍길동' 삼행시를 지어볼게요.

홍: 홍조 띤 하늘 아래,  
길: 길게 드리운 그림자 속에서,  
동: 동화 같은 꿈을 꾸는 우리.  

어떠신가요? 더 도와드릴 것이 있으면 말씀해 주세요!
-------------------------------------------------------------------------------

## 3-2. RunnableWithMessageHistory (top=k)

위의 RunnableWithMessageHistory를 사용하면 과거 대화가 무한정 누적될 수 있습니다. 이렇게 되면 대화를 여러 번할수록 입력 프롬프트의 길이가 무한정 늘어날 수 있기 때문에 일반적으로 최근 대화만 기억하도록 하기도 합니다.  

여기서는 "limited_chat_history"라는 세션 id를 사용하여 최대 몇 개까지의 대화만 기억할지 컨트롤 해보겠습니다.

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from typing import List
from langchain.chains import LLMChain
from langchain_core.runnables.history import RunnableWithMessageHistory

class LimitedChatMessageHistory(BaseChatMessageHistory):
    def __init__(self, max_conversations: int = 2):
        self.full_messages: List[BaseMessage] = []
        self.messages: List[BaseMessage] = []

        # max_converstion은 기억할 대화의 개수. 만약 -1을 해주지 않으면 max_conversations=2를 넣었을 때 과거 대화 2개랑 현재 대화 1개로 3개를 기억한다.
        # 하지만 -1을 해주면 max_conversations=2를 넣었을 때 과거 대화 1개, 현재 대화 1개 총 2개를 기억하게 된다.
        # 각 대화는 사용자 입력과 AI 응답으로 구성하므로 *2
        self.max_messages = (max_conversations -1) * 2

    def add_message(self, message: BaseMessage) -> None:
        self.full_messages.append(message)
        self.messages.append(message)
        if len(self.messages) > self.max_messages:
            self.messages.pop(0)

    def clear(self) -> None:
        self.messages = []
        self.total_messages = 0

    def get_messages(self) -> List[BaseMessage]:
        # session_history.messages로 출력할 경우 실제 호출한 것에서 과거 대화가 하나 빠지고 출력된다.
        # 따라서 -2를 추가하여 과거 대화 1개를 다시 포함하여 출력 하도록한다. -2인 이유는 사용자 질문, AI 답변 이렇게 2개가 한 쌍의 대화이기 때문이다.
        return self.full_messages[-self.max_messages-2:]

# 메시지 히스토리를 저장할 딕셔너리
session_id = "limited_chat_history"
message_store = {}
message_store[session_id] = LimitedChatMessageHistory(max_conversations=2)

chain_with_chat_history = RunnableWithMessageHistory(
    llm_chain,
    lambda session_id: message_store[session_id],
    input_messages_key="input",
    history_messages_key="history",
)

session_history = message_store[session_id]

# Example usage
result1 = chain_with_chat_history.invoke(
    {"input": "너는 누구"},
    config={"configurable": {"session_id": session_id}},
)

result2 = chain_with_chat_history.invoke(
    {"input": "홍길동으로 시를 지어볼래?"},
    config={"configurable": {"session_id": session_id}},
)

result3 = chain_with_chat_history.invoke(
    {"input": "더 감성적으로 지어볼래?"},
    config={"configurable": {"session_id": session_id}},
)

result4 = chain_with_chat_history.invoke(
    {"input": "현재 대화에서 내가 너한테 던진 첫번째 질문이 뭐였는지 알려줘"},
    config={"configurable": {"session_id": session_id}},
)

In [ ]:
session_history.get_messages()

[HumanMessage(content='더 감성적으로 지어볼래?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='물론이죠! 홍길동을 주제로 한 감성적인 시를 지어보겠습니다.\n\n---\n\n달빛 아래 홀로 서서  \n바람에 실려오는 옛 노래,  \n홍길동의 마음속 깊은 곳에  \n그리움이 피어오르네.\n\n자유를 꿈꾸며 떠난 길,  \n별빛이 길을 비추고,  \n그의 발걸음마다  \n희망의 꽃이 피어나네.\n\n세상은 여전히 어둡지만,  \n그의 눈에는 빛이 가득,  \n모두의 마음속에 남을  \n영원한 영웅, 홍길동이여.\n\n---\n\n어떠신가요? 더 도와드릴 것이 있으면 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 164, 'prompt_tokens': 469, 'total_tokens': 633, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f785eb5f47', 'finish_reason': 'stop', 'logprobs': None}, id='run-0f8e9e53-4268-48ea-8ef0-1eb3749d55c2-0', usage_metadata={'input_tokens': 469, 'output_tokens': 164, 'total_tokens': 633, 'input_token_details': {'a

In [ ]:
# 메시지 히스토리 출력
print("메시지 히스토리:")
# for message in session_history.messages:
for message in session_history.get_messages():
    print(f"{message.__class__.__name__}: {message.content}")
    print('--' * 50)

메시지 히스토리:
HumanMessage: 더 감성적으로 지어볼래?
----------------------------------------------------------------------------------------------------
AIMessage: 물론이죠! 홍길동을 주제로 한 감성적인 시를 지어보겠습니다.

---

달빛 아래 홀로 서서  
바람에 실려오는 옛 노래,  
홍길동의 마음속 깊은 곳에  
그리움이 피어오르네.

자유를 꿈꾸며 떠난 길,  
별빛이 길을 비추고,  
그의 발걸음마다  
희망의 꽃이 피어나네.

세상은 여전히 어둡지만,  
그의 눈에는 빛이 가득,  
모두의 마음속에 남을  
영원한 영웅, 홍길동이여.

---

어떠신가요? 더 도와드릴 것이 있으면 말씀해 주세요!
----------------------------------------------------------------------------------------------------
HumanMessage: 현재 대화에서 내가 너한테 던진 첫번째 질문이 뭐였는지 알려줘
----------------------------------------------------------------------------------------------------
AIMessage: 당신이 저에게 던진 첫 번째 질문은 "더 감성적으로 지어볼래?"였습니다. 더 궁금한 점이 있으면 언제든지 말씀해 주세요!
----------------------------------------------------------------------------------------------------


# 4. Text Splitter

## 4-1. RecursiveCharacterTextSplitter

이 텍스트 분할기는 일반적인 텍스트에 권장되는 방식입니다.  
이 분할기는 문자 목록을 매개변수로 받아 동작합니다.  

분할기는 청크가 충분히 작아질 때까지 주어진 문자 목록의 순서대로 텍스트를 분할하려고 시도합니다.  

기본 문자 목록은 ["\n\n", "\n", " ", ""]입니다.  

단락 -> 문장 -> 단어 순서로 재귀적으로 분할합니다.  
이는 단락(그 다음으로 문장, 단어) 단위가 의미적으로 가장 강하게 연관된 텍스트 조각으로 간주되므로, 가능한 한 함께 유지하려는 효과가 있습니다.  

텍스트가 분할되는 방식: 문자 목록(["\n\n", "\n", " ", ""]) 에 의해 분할됩니다.  

청크 크기가 측정되는 방식: 문자 수에 의해 측정됩니다.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

- chunk_size 매개변수를 3000 으로 설정하여 각 청크의 최대 크기를 3000자로 제한합니다.
- chunk_overlap 매개변수를 0으로 설정하여 인접한 청크 간에 중복은 없습니다.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [ ]:
!wget https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt

--2024-12-14 16:16:46--  https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43694449 (42M) [text/plain]
Saving to: ‘2016-10-20.txt’

2016-10-20.txt      100%[===================>]  41.67M  54.5MB/s    in 0.8s    

2024-12-14 16:16:49 (54.5 MB/s) - ‘2016-10-20.txt’ saved [43694449/43694449]



In [ ]:
with open("2016-10-20.txt") as f:
    file = f.read()  # 파일의 내용을 읽어서 file 변수에 저장합니다.

In [ ]:
file[:50]

' \n19  1990  52 1 22\n오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계'

In [ ]:
texts = text_splitter.create_documents([file])
len(texts)

25281

In [ ]:
texts[1]

Document(metadata={}, page_content='오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널

In [ ]:
texts[2]

Document(metadata={}, page_content='33분께 풀숲에 숨은 성씨가 허공에 난사한 10여발의 총알 중 일부를 왼쪽 어깨 뒷부분에 맞고 쓰러졌다  김 경위는 구급차가 도착했을 때 이미 의식이 없었고 심폐소생술을 하며 병원으로 옮겨졌으나 총알이 폐를 훼손해 오후 7시 40분께 사망했다  김 경위는 외근용 조끼를 입고 있었으나 총알을 막기에는 역부족이었다  머리에 부상을 입은 이씨도 함께 병원으로 이송됐으나 생명에는 지장이 없는 것으로 알려졌다  성씨는 오패산 터널 밑쪽 숲에서 오후 6시 45분께 잡혔다  총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다  총 때문에 쫓던 경관들과 민간인들이 몸을 숨겼는데 인근 신발가게 직원 이모씨가 다가가 성씨를 덮쳤고 이어 현장에 있던 다른 상인들과 경찰이 가세해 체포했다  성씨는 경찰에 붙잡힌 직후 나 자살하려고 한 거다 맞아 죽어도 괜찮다 고 말한 것으로 전해졌다  성씨 자신도 경찰이 발사한 공포탄 1발 실탄 3발 중 실탄 1발을 배에 맞았으나 방탄조끼를 입은 상태여서 부상하지는 않았다  경찰은 인근을 수색해 성씨가 만든 사제총 16정과 칼 7개를 압수했다 실제 폭발할지는 알 수 없는 요구르트병에 무언가를 채워두고 심지를 꽂은 사제 폭탄도 발견됐다  일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다')

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

In [ ]:
texts = text_splitter.create_documents([file])
len(texts)

25416

In [ ]:
texts[1]

Document(metadata={}, page_content='오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널

In [ ]:
texts[2]

Document(metadata={}, page_content='후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 33분께 풀숲에 숨은 성씨가 허공에 난사한 10여발의 총알 중 일부를 왼쪽 어깨 뒷부분에 맞고 쓰러졌다  김 경위는 구급차가 도착했을 때 이미 의식이 없었고 심폐소생술을 하며 병원으로 옮겨졌으나 총알이 폐를 훼손해 오후 7시 40분께 사망했다  김 경위는 외근용 조끼를 입고 있었으나 총알을 막기에는 역부족이었다  머리에 부상을 입은 이씨도 함께 병원으로 이송됐으나 생명에는 지장이 없는 것으로 알려졌다  성씨는 오패산 터널 밑쪽 숲에서 오후 6시 45분께 잡혔다  총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다  총 때문에 쫓던 경관들과 민간인들이 몸을 숨겼는데 인근 신발가게 직원 이모씨가 다가가 성씨를 덮쳤고 이어 현장에 있던 다른 상인들과 경찰이 가세해 체포했다  성씨는 경찰에 붙잡힌 직후 나 자살하려고 한 거다 맞아 죽어도 괜찮다 고 말한 것으로 전해졌다  성씨 자신도 경찰이 발사한 공포탄 1발 실탄 3발 중 실탄 1발을 배에 맞았으나 방탄조끼를 입은 상태여서 부상하지는 않았다  경찰은 인근을 수색해 성씨가 만든 사제총 16정과 칼 7개를 압수했다 실제 폭발할지는 알 수 없는 요구르트병에 무언가를 채워두고 심지를 꽂은 사제 폭탄도 발견됐다  일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다')

## 4-2. SemanticChunker

In [ ]:
!pip install langchain_experimental tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 3.2 MB/s eta 0:00:00


SemanticChunker는 LangChain의 실험적 기능 중 하나로, 텍스트를 의미론적으로 유사한 청크로 분할하는 역할을 합니다.

이 방법에서는 지정한 breakpoint_threshold_amount 표준편차보다 큰 차이가 있는 경우 분할됩니다.

breakpoint_threshold_type 매개변수를 "standard_deviation"으로 설정하여 청크 분할 기준을 표준편차 기반으로 지정합니다.

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import OpenAIEmbeddings

# OpenAI 임베딩을 사용하여 의미론적 청크 분할기를 초기화합니다.

text_splitter = SemanticChunker(OpenAIEmbeddings(),
    breakpoint_threshold_type="standard_deviation",
    breakpoint_threshold_amount=1.25)

<ipython-input-75-28be2501badb>:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  text_splitter = SemanticChunker(OpenAIEmbeddings(),


In [ ]:
script = '''허수현은 한반도 최북단 탄광마을이 낳은 수재였다. 그는 고등학교 졸업 직전 북한 전체에서 700명에게만 수여하는 ‘7.15 최우등상’을 수상했다. 7.15최우등상은 김정일이 평양 남산고급중학교를 졸업한 날인 1960년 7월 15일을 기념해 1987년에 만들어진 상이다.

지금은 이 상이 특권층 자식들을 대학에 보내기 위한 발판이 돼 각종 비리와 뇌물로 얼룩져 있지만, 상이 제정된 초기 몇 년은 정말 공부 잘하는 사람에게만 수여됐다. 상을 받게 되면 곧바로 중앙급 대학에 진학할 수 있었다. 북한에선 고등중학교 졸업생 중 20% 미만이 대학이나 전문학교에 갈 수 있다는 것을 감안하면 엄청난 특혜였다.

허 씨도 김책공업종합대학(김책공대)에 입학해 8년이나 공부했다. 그리고 그때 배운 지식을 활용해 지금은 한반도 최남단인 경남 마산의 해저터널 공사장에서 시공품질을 관리하는 공사차장으로 일하고 있다. 김책공대 졸업생이 네 번의 탈북을 반복한 뒤 건강이 악화돼 남의 등에 업혀 동남아 정글을 넘어 한국까지 오게 되고, 이후 남과 북에서 동시에 측량기사 자격을 받은 최초의 기술자가 돼 해저터널 공사장에서 일하게 되기까지 삶의 과정은 결코 순탄하지 않았다.


2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.

아버지가 사망하자 허 씨는 1년 정도 방황했다. 학교에도 나가지 않았다. 하지만 곧 마음을 다잡고, 부친의 소원대로 신분 상승을 하기로 결심했다. 3년을 열심히 공부해 고등학교 졸업반이 됐을 때 허 씨는 7.15 최우등상을 받을 정도로 공부를 잘하게 됐다. 북한도 대도시의 교육 환경이 매우 좋기 때문에 외진 탄광마을에서 수상자를 배출한다는 것은 하늘의 별 따기였다.

하지만 상을 받아도 문제였다. 평양에서 대학을 다닌다는 것은 가족들에겐 엄청난 희생이 필요한 일이었다. 그런데 그가 김책공대에 입학한 1992년엔 탄광마을에 배급이 제대로 공급되지 않았다. 평양에서 대학을 다닐 돈이 나올 리 만무했다.

허 씨는 대학 대신에 군대에 가려고 시도했다. 대학을 갈 사정이 못되니 군에 입대해 노동당원이 되면 신분이 그나마 좀 바뀔 것이라고 생각했던 것이다.

하지만 7.15 최우등상 수상자는 군대에 보내지 않는다는 지침이 있어 결국 갈 수는 없었다. 그는 울며 겨자 먹기로 김책공대 지질탐사학부에 입학했다.

그해 온성에서 중앙대학에 입학한 사람은 단 두 명이었다. 허 씨 외에 이과대학 입학생이 한 명 더 있었다. 온성군은 그런 동네였다.

● 돈에 성적을 뺏기던 시절
북한의 다양한 산업현장에서 활약하는 인재들을 키우는 김책공대는 학제가 길어 7년을 다녀야 졸업증을 받을 수 있다. 그런데 대학을 다니며 각종 공사현장에 끌려 다니다 보니 진도가 밀려 7년 안에 졸업하기가 어렵다. 허 씨도 학제보다 1년을 더 다녀 2000년에야 졸업할 수 있었다.

그가 대학을 다니던 1990년대 중반은 북한에서 고난의 행군 시기라 사방에서 아사자가 속출할 때였다. 대학 기숙사에서 주는 밥을 먹으면 굶어죽을 수밖에 없었다.

탄광마을에서 태어나 김책공대에 입학한 허 씨와, 어촌마을에서 태어나 김일성대에 입학한 기자는 평양에서 대학을 다닌 시기가 정확히 겹친다. 그래서 인터뷰 내내 떠올리기 싫은 추억들이 소환됐다. 몇 개를 소개하면 이런 식이다.

“1993년 공화국 창건 행사 때 김일성대 학생들은 깃발을 들고 김일성광장을 통과했습니다. 그 연습만 3개월 하면서 너무 힘들어 죽을 뻔 했죠.”

“김책공대는 촛불을 들고 김일성대를 따라갔죠. 저도 죽을 뻔 했어요.”

“제대군인인 학급 소대장, 청년단체비서 이런 사람들은 가난한데 공부 잘하는 학생들을 곁에 한둘씩 끼고 있죠. 그리곤 시험 때마다 자기 것도 좀 써달라고 사정하죠. 그걸 거절하기 어려워 저는 시험 칠 때마다 시험지 3개를 써주었어요. 다양한 볼펜을 준비해서 한 장은 정자로 쓰고, 다른 장은 흘겨 쓰고, 이런 식으로 답을 적어 교수가 뒤돌아서 있을 때 공부 못하는 제대군인들에게 몰래 건네주었죠. 대신 그들은 각종 비용을 걷을 때 나를 빼주기도 했고, 가끔 도시락도 두 개를 갖고 와서 허기진 배도 채워주었습니다.”

“저도 그랬어요. 국가졸업시험 때까지 남의 시험지를 작성해주었어요. 대신 저는 돈을 받았습니다. 방학 때 집에 가지도 않았지요. 온성까지 기차로 일주일 넘게 걸리는데 왔다갔다 시간 낭비가 크고, 또 가봐야 집에서 보태줄 수도 없으니 방학 때는 제대군인들 과외를 해주고 돈을 받았습니다.”

“저는 졸업장을 받고 나니, 3점짜리 과목이 몇 개 있었어요. 저는 대학 내내 3점을 받은 적이 없거든요. 5점 만점에 3점은 낙제를 겨우 면한 수준인데, 대학 교무부에 찾아가 싸우지도 않았어요. 어차피 이 체제가 싫어서 탈북하려 결심한 마당에 3점이 대수냐고 생각했죠.”

“저도 졸업증에 받지 않은 3점들이 있었어요. 권력자 부모를 둔 학생들은 좋은 곳에 가기 위해 대학 교무부 직원들에게 뇌물을 주고 컴퓨터에서 점수를 바꾸었어요. 5점 최우등생과, 4점 우등생, 3점 보통생의 전체 비율을 바꿀 수 없으니 5점으로 조작하려면 누군가의 점수를 빼앗아야 했죠. 제일 힘이 없는 우리가 점수를 빼앗긴 거였죠.”

최우등을 하고도, 돈과 권력이 없으면 3점 졸업증을 받아야 했던 시대. 허 씨와 기자는 그 시대에 평양에서 대학을 다녔다. 기숙사에서 밥을 몇 숟가락만 주던 때라 대학 시절을 떠올리면 배고팠던 기억밖에 없다. 졸업증을 받은 것만으로도 기적이었다. 뇌물로 성적을 조작하고, 남의 졸업논문을 베껴 써서 제출하는 상황은 지금도 별반 나아지진 않았다.


2018년 뉴질랜드로 한 달 동안 어학연수를 떠났던 시기의 모습.


● 북한의 측량기사들
아버지가 없는 가난한 탄광마을 출신의 김책공대 졸업생에게 좋은 직업이 기다릴 리 만무했다. 북한은 직업 선택의 자유가 없다. 대학을 졸업하면 중앙에서 어디로 가라고 임명한다.
권력과 부를 가진 집안에서 태어나면 대학 때 실컷 놀고도 중앙당이나 외화벌이 업체, 보위부 등 권력기관에 발령을 받는다. 가난한 자들의 운명은 그와 반대이다.

허 씨는 2000년 3월 대학을 졸업하면서 졸업증과 함께 측량기사 자격을 부여받았다. 그리고 평양 인근 대동군 시정노동자구에 있는 중앙측량단으로 발령을 받았다. 모두가 기피하는 직업이었다.

측량기사는 20㎏이 넘는 장비를 메고 매일 같이 산을 오르내려야 했다. 1000분의 1 오차 범위 내에서 지도에 점 하나를 찍는데 사나흘이 걸렸고, 한 구역을 측량하는데 2~3개월이 걸렸다. 깊은 산속에 천막을 치고 야인생활을 하기 일쑤였다. 그나마 현장기사는 배급을 받을 수 있어 다행이었다.

북한은 측량을 하는 기관이 두 개가 있다. 하나는 중앙측량단이고, 다른 하나는 인민무력부(군) 측지국이다. 그런데 진짜 좌표는 측지국이 갖고 있다. 중앙측량단의 좌표는 일부러 정확한 좌표와 다르게 작성한다. 좌표가 고급 비밀이기 때문에 외부에 정보가 새나가면 안된다는 이유 때문이다.

허 씨는 한국에 온 뒤 큰 허탈감을 느꼈다. 위성으로 GPS를 찍는 시대를 보니 그 무거운 장비를 메고 다니며 점 하나를 찍겠다고 며칠씩 바친 과거가 너무 허망하게 생각됐기 때문이다. 북한도 2005년경부터 러시아 위성항법체계인 ‘글로나스’의 도움을 받아 위성 측량을 시도했다고는 알려졌지만, 어느 정도 활용되는지는 아직 파악되지 않고 있다.

● 원시적 석탄채굴
허 씨는 2001년 말에 온성으로 돌아왔다. 뇌물이 없으면 이직도 힘든 세상이지만, 아버지도 없는데 어머니마저 아파서 쓰러졌다고 하니 집으로 보내준 것이다. 실제로 어머니를 간호할 사람은 허 씨 밖에 없었다.

새로 발령받은 직장은 풍인탄광 기술과였다. 그러나 할 일은 없었다. 1990년대 중반 고난의 행군 시기 온성 탄광들은 모두 침수됐다. 전기가 없어 물을 빼낼 수가 없었기 때문이다. 그렇게 몇 년을 방치하다보니 갱을 다시 사용할 수가 없었다.

탄광에 다니던 사람들은 먹고 살기 위해 다른 방법을 찾았다. 일제 때 했던 방식대로 얇은 탄층 채굴을 시작한 것이다. 삽과 곡괭이로 수직굴을 파들어 가는데, 운이 좋으면 8m에서 탄층을 만나기도 하지만, 운이 나쁘면 25m까지 파들어 간다. 탄층을 만나면, 그걸 따라 이번엔 가로로 파 들어간다. 탄층의 두께는 보통 0.8~1.2m였다. 양동이를 매단 도르래를 타고 수직굴에 들어가 몸을 돌리기도 어려운 좁은 공간에서 석탄을 파서 다시 양동이로 끌어올렸다. 도무지 현대인이라고 볼 수 없는 원시적 채탄방식이었지만, 그렇게 해서라도 석탄을 캐서 팔아야 장마당에서 먹을 것을 사올 수 있었다. 이런 수직굴을 온성에선 ‘노두’라고 불렀다. 온성에는 이런 노두가 수없이 많았다. 노두당 가족, 친척, 친구 등 5~7명이 팀을 이뤄 작업했는데, 이런 사람들을 ‘노두공’이라 불렀다. 북쪽은 날씨가 춥기 때문에 먹는 것 못지않게 석탄도 귀하다. 캐내면 파는 것은 큰 문제가 안됐다.

노두도 1년 내내 할 수 있는 것이 아니었다. 땅이 어는 1~3월에만 할 수 있었고, 봄이 와서 땅이 녹으면 수직갱이 버틸 수 없기 때문에 버려야 했다. 구멍을 방치해서도 안됐다. 단속기관 사람들이 찾아와 갱을 다시 메웠는지 조사한다. 단속할 권한이 있다는 것은 뇌물을 받을 수 있다는 것을 의미하기 때문에 얼렁뚱땅 넘어가지 않는다.

온성 전체에 이런 노두가 수없이 생겨났다 사라졌다. 겨울이면 할 일이 없는 농민들도 이 일에 매달렸다. 안전은 뒷전이라 붕괴와 추락, 가스질식 등으로 죽는 사람들이 계속 생겨났지만 큰 문제가 되진 않았다. 그 일을 하지 않으면 어차피 굶어죽기 때문이다.

허 씨가 소속된 기술과는 늘 각종 공사에 동원됐다. 도로 공사, 발전소 공사 등등 인력을 차출하는 공사는 끊이질 않았다. 김책공대에서 배운 지식을 쓸 곳은 없었다.

이렇게 살다가 어느새 결혼할 나이가 됐다. 그래도 허 씨는 명색이 탄광마을이 배출한 수재이고, 평양에서 김책공대까지 나왔던 터라 여성들에게 나름 인기가 있었다.

32살 때인 2006년 그는 10살 어린 여성과 결혼했다. 아내는 탄광 선전대 가수로 나름 인기가 좋았다. 이성적인 지식인과 감성적인 예술인의 조합이었다.

그렇지만 매력과 결혼생활은 별개였다. 결혼한 직후부터 둘은 다투는 일이 많았다. 서로가 서로를 견디기 어려워했다. 4년간의 결혼생활 끝에 둘은 이혼하기로 합의했다. 어린 딸은 아내가 키우기로 했다. 이때부터 그는 중국으로 눈을 돌렸다.


2022년 인천지하철 공사 현장에서 일하고 있는 허 씨.


● 두만강을 넘나들다
강 건너 연변 왕청에는 아버지의 삼촌과 사촌 등 친척들이 살고 있었다. 국경 근처라 많은 사람들이 중국을 드나들 때도 허 씨는 명색이 김책공대 졸업생인데 조국을 배반하면 안 된다는 마음이 컸다. 하지만 이혼 후 세상이 달리 보이기 시작했다. 이혼까지 한 마당에 눈치 볼 일도, 무서울 일도 없었다.

2010년 겨울 그는 국경경비대에 돈을 쥐어주고 두만강을 넘었다. 탈북한 것은 아니고, 친척에게 도움만 받자는 목적이었다. 저녁에 넘어가서 친척에게 전화를 하고 새벽이 되기 전에 다시 북으로 넘어왔다.

2012년 2월 그는 두 번째로 중국으로 갔다. 당시는 김정일이 사망한지 얼마 안됐던 때라 경계가 매우 심했다. 그는 북한군 군복을 입고 길을 떠났다. 군인은 초소에서 잘 단속하지 않았기 때문이다. 강을 따라 난 도로로 한참을 가다가 어둠이 내릴 때 바로 두만강을 넘었다. 중국 부락의 아무 집이나 들어가 왕청 친척에게 전화를 좀 하고 싶다고 했다. 전화를 하고 몇 시간 정도 머물렀는데 갑자기 공안이 들이닥쳤다. 집주인이 그를 도와주는 척하고는 신고를 했던 것이다. 알고 보니 중국은 그즈음부터 탈북자를 신고하면 포상금을 주는 제도를 운영하고 있었다. 특히 북한군 국경경비대가 수시로 건너와 사람까지 죽이며 노략질을 했기 때문에, 중국에서 북한군은 최고의 기피대상이었다.

탈북자들이 북송 전 대기하는 도문변방수용소에 끌려갔는데 며칠 뒤 보위부에서 차를 갖고 건너와 그를 싣고 갔다.

그렇지만 그는 예상과는 달리 20일 정도 가수감됐다가 석방됐다. 서류를 보니 중국에 친척이 다 있는 것도 확실하니, 도움 좀 받으려 넘어갔을 뿐이라는 그의 말이 인정됐다. 탈북은 배반이지만, 그의 경우엔 일탈 정도로 간주됐다.

보위부라고 해봐야 어차피 한동네 사는 아는 사람들이었는데, 그들은 지역이 배출한 인재의 경력을 망가뜨리고 싶지 않았는지 그다지 혹독하게 대하진 않았다.

하지만 허 씨 입장에선 아무 것도 이루지 못하고 잡혀오니 화가 났다. 보위부 감방에서 그는 강 건너편에서 일하다 잡혀왔다는 사람을 알게 됐다. 그는 자기가 일했던 중국 훈춘의 한 슈퍼의 이름을 알려주면서, 다시 건너가 자기 이름을 대면 사장이 고용해 줄 것이라고 했다.

감방에서 나온 허 씨는 다시 두만강을 넘었다. 세 번째 탈북이었다. 그는 감옥 동기가 알려준 슈퍼로 갔다. 왕청에 가지 않은 이유는 친척들은 별로 도와줄 의향이 없어 보였기 때문이다.
슈퍼는 중국과 나진선봉을 연결하는 도로 옆에 있었는데, 두만강 건너 허 씨네 동네가 약 10㎞ 밖에 빤히 바로 보였다. 허 씨가 내륙 깊이 들어가지 않은 이유는 딸을 데려오고 싶어서였다. 고향 가까이 있어야 집과 연락이 수월하다고 판단했다. 허 씨가 일하는 슈퍼에는 북한을 오가는 운전기사들이 수시로 드나들었다. 이들을 통해 그는 전처에게 휴대전화를 전달할 수 있었다.

● 자수해 감옥에 가다
그렇게 1년쯤 지났는데 사고가 생겼다. 중국 휴대전화를 받은 전처가 그걸 이용해 돈 벌려고 브로커 일을 하다가 보위부에 체포된 것이다. 전 남편마저 실종되니 보위부는 그녀에게 한국행을 기도했다는 누명을 씌웠다.

그 소식이 허 씨에게도 전달됐다. 그래도 4년 간 살았던 정도 있고, 어린 딸까지 있으니 모르는 척 할 수가 없었다. 그는 다시 북에 나가기 위해 자수하기로 결심했다. 자신이 나타나야 전 남편이 한국에 갔다는 누명을 벗길 수 있을 것 같았다.

슈퍼 주인은 전과자 출신이었는데, 그가 북으로 가겠다고 하자 감옥에서 나오는 자기의 비법을 전수해주었다. 폐 주변에 황산철을 주사하면 고열이 나고, 염증이 생긴다면서, 자신은 그 방법으로 병원에 호송됐다가 도망쳤다고 했다. 단 이 방법의 단점은 한 달 안에 황산철을 세척하지 못하면 생명을 잃을 수도 있다고 했다.

허 씨는 그의 조언에 따라 폐에 황산철을 네 군데나 주입했다. 그리고 스스로 공안에 자수한 뒤 2014년 4월 13일 북송됐다. 그런데 이번은 보위부도 호락호락하지 않았다. 불행하게도 그가 자수한 뒤 공안이 그의 숙소를 뒤져 소지품을 북송할 때 함께 보냈던 것이다. 그 속에는 한국 영사관, 한국인 전도사 등의 전화번호가 적힌 수첩이 있었다. 이게 화근이 됐다. 고문이 시작됐다.

중국 사장이 알려준 방법은 확실히 효과가 있었다. 폐가 곪아가기 시작했던 것이다. 염증으로 심한 열까지 나 거동을 못할 정도가 됐다. 보위부는 감방에서 죽이긴 싫었는지 그를 가석방으로 집에 보냈다.

그때가 5월 말이었다. 중국 사장이 당부한 폐를 씻어야 하는 한 달이 지난 것이다. 집에 가서 이러저런 치료를 해봤지만 점점 악화됐다. 이렇게 지내다간 죽을 것 같았다. 그는 황산철 주사를 맞은 지 네 달이 지난 8월 23일 다시 중국으로 넘어왔다. 이번엔 하얼빈에 들어가 식당에 취직해 치료에만 전념했다. 하지만 몸은 점점 더 쇠약해졌다.

몸을 운신하기 어려워지자 허 씨는 죽더라도 고향에 가서 죽겠다고 생각했다. 그해 11월말 그는 다시 연길에 왔다. 연길에서 혹시나 도움을 받지 않을까 싶어 처음으로 교회에 찾아갔는데, 거기서 집도 제공하고 치료도 해주었다. 12월 말 허 씨는 산소 호흡기에 의지하는 신세가 됐다. 쇼크도 수시로 왔다. 교회에선 한국에 가서 치료를 받아야 살 수 있다며 한국행선을 주선해주었다.


교회 목회자가 될 꿈을 꾸던 당시의 허 씨. 2017년 뉴욕에서 찍은 사진이다.


● 최초로 남북 측량기사 자격 모두 획득
2015년 2월 그는 여러 탈북민들과 함께 한국으로 떠났다. 동남아 정글에서 일행을 따라갈 수 없을 때 친한 동생이 그를 업고 산을 넘었다. 우여곡절 끝에 한국에 도착했지만 다음날 적십자병원에 실려갔다. 이곳에서 4개월 동안 치료를 받다보니 하나원도 나오지 못했다.

허 씨는 인천에 거주지가 배정됐다. 건강이 너무 악화돼 일을 할 수 없는 상태라 2016년말까지 병원에 다니며 통원치료를 받았다. 하나원을 나온 다른 사람들이 하나둘 취직해 일자리를 얻는 것을 보고 조급한 마음에 노량진에서 공무원 시험을 준비하기도 했지만, 건강이 악화돼 다시 병원에 실려 가기도 했다. 한국의 의술은 역시 대단했다. 2년이라는 오랜 시간이 걸리긴 했지만, 끝내 허 씨를 완치시켰다.

치료 받는 기간에 허 씨는 목회자의 길을 걸어볼까 싶어 1년 남짓 성경공부도 했고, 화장품 회사에 취직해 일도 해봤지만 적성에 맞지 않았다.

많은 고민 끝에 북에서 배운 측량기사 일을 다시 해보려 알아보니, 남쪽도 측량기사는 일도 힘들고 보상도 적었다. 그렇지만 적성에 맞지 않는 곳에서 시간을 낭비하는 것보다는 그래도 자신있는 분야를 파보자는 생각에 2018년 한 엔지니어링 회사에 취직했다. 취직은 했어도 아무런 건설기술인 등급도 받지 못했기 때문에 회사에서 가장 많은 나이임에도 허드레 일만 해야 했다. 연봉은 2300만 원이었다. 그는 앞으로 살아가려면 자격증이 필수라는 것을 깨달았다. 어떤 자격을 갖고 있고, 어떤 경력을 쌓는가에 따라 연봉도 결정됐다.

허 씨는 회사에 다니며 1년 동안은 토목계측에 대한 용어부터 수첩에 적어 기본적인 지식을 배웠다. 그리고 이듬해 대구과학대에 입학했다. 일도 하면서 대학 공부까지 하려니 야간반을 다닐 수밖에 없었는데, 전국의 측량 관련 야간반 중에 대구가 그나마 가까웠다.

가깝다고 해도 당시 일을 하던 포항의 건설현장에서 150㎞나 떨어져 있었다. 일주일에 서너번씩 왕복 300㎞를 달려 수업을 듣고 오는 일은 결코 쉽지 않았다.

돌아오는 길에 너무 피곤해 졸음 쉼터에서 쪽잠을 자기 일쑤였다. 잠깐 눈을 붙인다는 것이 해가 중천에 걸릴 때까지 잔적도 있다.

허 씨는 대학 공부와 병행해 자격증 시험도 준비했다. 3년 동안 주경야독의 삶을 끈질기게 이어나간 끝에 그는 2022년 대학을 우수한 성적으로 졸업하고, 동시에 측량 및 지형공간정보기사와 토목기사 자격증을 받을 수 있었다. 허 씨는 남과 북에서 측량기사 자격을 각각 받은 최초의 사례다.

이러한 자격증과 경력을 인정받아 현재 허 씨는 건설기술인협회에서 고급기술인으로 인정받고 있으며, 300억 원 규모 이상의 토목공사를 책임지고 할 수 있다.


2019년 여수에서 전력 케이블용 해저터널 공사장에서 작업하는 모습. 뒷모습이 보이는 사람이 허 씨이다.


● “배워야 살 수 있다”
허 씨는 김책공대에서 무려 8년이나 공부를 했지만 여기선 모든 것을 다시 배워야 했다고 말했다.

“남쪽에 오니 용어는 물론, 장비나 자재 등 모든 것이 북한과 달랐습니다. 비유하면 북에서 통나무로 집을 짓는 법을 배웠는데, 남쪽에 오니 시멘트로 아파트를 짓는 격이죠. 그럼 집 짓는 법을 처음부터 다시 배워야하죠. 물론 북한 김책공대 과정이 전혀 의미 없는 것은 아닙니다. 북한에선 가장 기본적인 것, 즉 공부하는 방법을 배운 것 같습니다.”

그는 남북은 통합성에서도 차이가 크다고 했다.

“여기는 한 가지를 하려면 열 가지를 알아야 합니다. 많은 것들이 서로 연결돼 있어 전체적으로 진행되죠. 반면 북한은 직무가 매우 세분화돼 있고, 맡겨진 것만 하면 됐어요.”

자격증과 경력을 쌓고 나니 연봉도 빠르게 올라갔다. 김책공대까지 입학했던 북한 시골 수재가 다시 자신의 두뇌와 재능을 발휘하는 데는 오랜 시간이 걸리지 않았다. 2300만 원에서 시작했던 연봉은 6년 만에 3배 이상 높아졌다.

그는 여기에 만족하지 않았다. 그는 올해 3월 다시 대구대 공간정보전문기술 석사과정에 입학했다. 토질 및 기초기술사 자격을 취득하기 위해서다. 다른 자격증과는 달리 이 자격은 받기가 매우 어렵다. 허 씨의 계획은 향후 5년 안에 석사와 함께 기술사 자격을 획득하는 것이다.

그렇다고 공부만 할 수는 없는 일. 요즘 허 씨는 마산만에서 스팀배관 부설을 위한 해저터널을 뚫는 작업을 하고 있다. 그의 직책은 공사차장. 터널 시공 품질을 총괄하는 중요한 자리다.

건설 현장은 매우 거칠고 다툼도 많다. 외국인 근로자들도 많아 의사소통의 문제도 많다. 많은 어려움이 있지만 허 씨는 포기하지 않고 한 걸음씩 나가고 있다.

“거친 현장이라서 그런지 북에서 왔다고 우습게 보는 사람들이 많아요. 아직도 저는 ‘나는 여전히 이방인이구나’라는 생각을 하고 있습니다. 그렇지만 실력은 남에게 뒤진다고 생각하지 않습니다. 여러 곳에서 이직 제안도 많이 받습니다.”

2023년 남북하나재단이 주관한 정착사례 발표대회에서 그는 최우수상인 통일부 장관상을 받았다. 하지만 허 씨는 지금은 정착의 첫 발자국을 뗀 것에 불과하다고 했다.

그는 자신의 이름으로 회사를 만들어 키우는 것을 다음 목표로 정했다. 그 목표가 달성되면 또 할 일이 있다. 나아가 통일이 되면 할 계획도 미리 생각해두었다.

“저는 북에 돌아가 여기서 배운 기술을 북에 전수할 겁니다. 한국은 토목 공사를 할 때 측량, 시공, 품질까지 다 알아야 합니다. 통일 되면 교통인프라를 다시 정리해야 할 것인데, 북한에서 대학을 다녔던 동창들은 통합성에 있어 크게 떨어집니다. 이런 것을 가르쳐야죠. 그리고 남북에서 모두 측량기사 자격을 받은 유일한 사람이니 다른 꿈도 있습니다. 남북공간정보 통합지도체계를 만드는 것도 중요한 목표입니다.”

꿈을 말할 때 그는 가장 행복한 표정이었다.'''

In [ ]:
chunks = text_splitter.split_text(script)

In [ ]:
len(chunks)

29

In [ ]:
chunks[0]

'허수현은 한반도 최북단 탄광마을이 낳은 수재였다. 그는 고등학교 졸업 직전 북한 전체에서 700명에게만 수여하는 ‘7.15 최우등상’을 수상했다. 7.15최우등상은 김정일이 평양 남산고급중학교를 졸업한 날인 1960년 7월 15일을 기념해 1987년에 만들어진 상이다. 지금은 이 상이 특권층 자식들을 대학에 보내기 위한 발판이 돼 각종 비리와 뇌물로 얼룩져 있지만, 상이 제정된 초기 몇 년은 정말 공부 잘하는 사람에게만 수여됐다. 상을 받게 되면 곧바로 중앙급 대학에 진학할 수 있었다. 북한에선 고등중학교 졸업생 중 20% 미만이 대학이나 전문학교에 갈 수 있다는 것을 감안하면 엄청난 특혜였다. 허 씨도 김책공업종합대학(김책공대)에 입학해 8년이나 공부했다. 그리고 그때 배운 지식을 활용해 지금은 한반도 최남단인 경남 마산의 해저터널 공사장에서 시공품질을 관리하는 공사차장으로 일하고 있다. 김책공대 졸업생이 네 번의 탈북을 반복한 뒤 건강이 악화돼 남의 등에 업혀 동남아 정글을 넘어 한국까지 오게 되고, 이후 남과 북에서 동시에 측량기사 자격을 받은 최초의 기술자가 돼 해저터널 공사장에서 일하게 되기까지 삶의 과정은 결코 순탄하지 않았다. 2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.'

In [ ]:
chunks[1]

'● 신분 상승의 꿈\n그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다.'

In [ ]:
chunks[2]

'세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다. 온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다. 탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다. 부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다. 아버지가 사망하자 허 씨는 1년 정도 방황했다. 학교에도 나가지 않았다.'

In [ ]:
chunks[3]

'하지만 곧 마음을 다잡고, 부친의 소원대로 신분 상승을 하기로 결심했다. 3년을 열심히 공부해 고등학교 졸업반이 됐을 때 허 씨는 7.15 최우등상을 받을 정도로 공부를 잘하게 됐다. 북한도 대도시의 교육 환경이 매우 좋기 때문에 외진 탄광마을에서 수상자를 배출한다는 것은 하늘의 별 따기였다. 하지만 상을 받아도 문제였다. 평양에서 대학을 다닌다는 것은 가족들에겐 엄청난 희생이 필요한 일이었다. 그런데 그가 김책공대에 입학한 1992년엔 탄광마을에 배급이 제대로 공급되지 않았다. 평양에서 대학을 다닐 돈이 나올 리 만무했다. 허 씨는 대학 대신에 군대에 가려고 시도했다. 대학을 갈 사정이 못되니 군에 입대해 노동당원이 되면 신분이 그나마 좀 바뀔 것이라고 생각했던 것이다. 하지만 7.15 최우등상 수상자는 군대에 보내지 않는다는 지침이 있어 결국 갈 수는 없었다. 그는 울며 겨자 먹기로 김책공대 지질탐사학부에 입학했다. 그해 온성에서 중앙대학에 입학한 사람은 단 두 명이었다. 허 씨 외에 이과대학 입학생이 한 명 더 있었다.'

# 5. Embedding

## 5-1. OpenAI Embedding

In [ ]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from langchain.embeddings import OpenAIEmbeddings

data = ['저는 배가 고파요',
        '저기 배가 지나가네요',
        '굶어서 허기가 지네요',
        '허기 워기라는 게임이 있는데 즐거워',
        '스팀에서 재밌는 거 해야지',
        '스팀에어프라이어로 연어구이 해먹을거야']

df = pd.DataFrame(data, columns=['text'])
df

,text
0,저는 배가 고파요
1,저기 배가 지나가네요
2,굶어서 허기가 지네요
3,허기 워기라는 게임이 있는데 즐거워
4,스팀에서 재밌는 거 해야지
5,스팀에어프라이어로 연어구이 해먹을거야


In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
query_result = embeddings.embed_query('안녕하세요')

In [ ]:
len(query_result)

1536

In [ ]:
print(query_result)

[-0.01363905893128265, -0.009446358031268955, -0.005943919478104129, -0.02294480773314062, -0.012475839748973755, 0.018202455112672995, -0.019774717717347474, 0.005880006530671005, -0.01290405817315644, 0.008788053555120605, 0.012060404659335807, -0.0032292101440946344, -0.02400576489570482, -0.014495493917002743, -0.005608375572757586, -0.00717105160784615, 0.027226986387031634, -0.013421754351216444, 0.007886878761139214, -0.017435497880830222, -0.004291765689138248, -0.015876016748055204, 0.0029112423924244645, -0.012469449013024025, 1.494218680647895e-05, 0.00395302595015552, 0.00401374352961378, -0.023916287141827502, 0.014201493986281313, -0.02166654505918757, 0.002701926930787398, 0.006953747027779942, -0.01880323830866059, -0.005582810766313392, 0.010194142123939902, -0.012322448116340673, 0.0024031330167810336, -0.014802277182268907, 0.005307983974763789, -0.007439486732123384, 0.018125758830695136, -0.014738364234835783, 0.009280183995413775, -0.014252624530492341, -0.0010545

In [ ]:
def get_embedding(text):
  return embeddings.embed_query(text)

In [ ]:
df['embedding'] = df.apply(lambda row: get_embedding(
        row.text
    ), axis=1)

In [ ]:
df

,text,embedding
0,저는 배가 고파요,"[-0.016637360179694076, -0.02178889820399648, ..."
1,저기 배가 지나가네요,"[-0.0032914344795325024, -0.027514765824463994..."
2,굶어서 허기가 지네요,"[-0.006181030746934311, -0.006950793503402665,..."
3,허기 워기라는 게임이 있는데 즐거워,"[-0.011329255872108568, -0.011715852525143846,..."
4,스팀에서 재밌는 거 해야지,"[-0.01610845668785982, -0.014401600417918041, ..."
5,스팀에어프라이어로 연어구이 해먹을거야,"[-0.0021389091187001565, -0.030034279922246478..."


벡터의 유사도를 구하는 방법으로는 코사인 유사도가 있다.  
코사인 유사도: https://wikidocs.net/24603  

In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def return_answer_candidate(df, query):
    query_embedding = get_embedding(
        query
    )
    df["similarity"] = df['embedding'].apply(lambda x: cos_sim(np.array(x),
                                                            np.array(query_embedding)))
    results_co = df.sort_values("similarity",
                                ascending=False,
                                ignore_index=True)
    return results_co.head(3)

In [ ]:
sim_result = return_answer_candidate(df, '아무 것도 안 먹었더니 꼬르륵 소리가나네')
sim_result

,text,embedding,similarity
0,굶어서 허기가 지네요,"[-0.006181030746934311, -0.006950793503402665,...",0.838333
1,스팀에어프라이어로 연어구이 해먹을거야,"[-0.0021389091187001565, -0.030034279922246478...",0.821603
2,저는 배가 고파요,"[-0.016637360179694076, -0.02178889820399648, ...",0.814259


## 5-2. 벡터 저장소

In [ ]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.3 MB/s eta 0:00

크로마는 오픈소스 벡터 데이터베이스입니다. Chroma는 Apache 2.0 라이선스가 부여됩니다.

- 공식 도큐먼트
- 홈페이지

**Apache 2.0 License 에 대해**

Apache 2.0 라이선스는 오픈 소스 라이선스 중 하나로, 누구나 소프트웨어를 자유롭게 사용, 수정, 배포할 수 있도록 허용하는 라이선스입니다.

이 라이선스의 주요 특징은 다음과 같습니다:

- 상업적 사용 가능: Apache 2.0 라이선스 하에 배포된 소프트웨어는 상업적 목적을 포함하여 어떤 목적으로든 사용할 수 있습니다. 이는 기업이나 개인이 이 라이선스로 된 소프트웨어를 상업적 제품이나 서비스에 통합하는 것을 가능하게 합니다.

- 소스 코드 공개 의무 없음: 소프트웨어를 수정하거나 확장한 경우, 그 변경된 소스 코드를 공개할 의무가 없습니다. 하지만 원본 소프트웨어가 Apache 2.0 라이선스를 따른다면, 변경된 버전 역시 Apache 2.0 라이선스 하에 배포되어야 합니다.

In [ ]:
script = '''허수현은 한반도 최북단 탄광마을이 낳은 수재였다. 그는 고등학교 졸업 직전 북한 전체에서 700명에게만 수여하는 ‘7.15 최우등상’을 수상했다. 7.15최우등상은 김정일이 평양 남산고급중학교를 졸업한 날인 1960년 7월 15일을 기념해 1987년에 만들어진 상이다.

지금은 이 상이 특권층 자식들을 대학에 보내기 위한 발판이 돼 각종 비리와 뇌물로 얼룩져 있지만, 상이 제정된 초기 몇 년은 정말 공부 잘하는 사람에게만 수여됐다. 상을 받게 되면 곧바로 중앙급 대학에 진학할 수 있었다. 북한에선 고등중학교 졸업생 중 20% 미만이 대학이나 전문학교에 갈 수 있다는 것을 감안하면 엄청난 특혜였다.

허 씨도 김책공업종합대학(김책공대)에 입학해 8년이나 공부했다. 그리고 그때 배운 지식을 활용해 지금은 한반도 최남단인 경남 마산의 해저터널 공사장에서 시공품질을 관리하는 공사차장으로 일하고 있다. 김책공대 졸업생이 네 번의 탈북을 반복한 뒤 건강이 악화돼 남의 등에 업혀 동남아 정글을 넘어 한국까지 오게 되고, 이후 남과 북에서 동시에 측량기사 자격을 받은 최초의 기술자가 돼 해저터널 공사장에서 일하게 되기까지 삶의 과정은 결코 순탄하지 않았다.


2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.

아버지가 사망하자 허 씨는 1년 정도 방황했다. 학교에도 나가지 않았다. 하지만 곧 마음을 다잡고, 부친의 소원대로 신분 상승을 하기로 결심했다. 3년을 열심히 공부해 고등학교 졸업반이 됐을 때 허 씨는 7.15 최우등상을 받을 정도로 공부를 잘하게 됐다. 북한도 대도시의 교육 환경이 매우 좋기 때문에 외진 탄광마을에서 수상자를 배출한다는 것은 하늘의 별 따기였다.

하지만 상을 받아도 문제였다. 평양에서 대학을 다닌다는 것은 가족들에겐 엄청난 희생이 필요한 일이었다. 그런데 그가 김책공대에 입학한 1992년엔 탄광마을에 배급이 제대로 공급되지 않았다. 평양에서 대학을 다닐 돈이 나올 리 만무했다.

허 씨는 대학 대신에 군대에 가려고 시도했다. 대학을 갈 사정이 못되니 군에 입대해 노동당원이 되면 신분이 그나마 좀 바뀔 것이라고 생각했던 것이다.

하지만 7.15 최우등상 수상자는 군대에 보내지 않는다는 지침이 있어 결국 갈 수는 없었다. 그는 울며 겨자 먹기로 김책공대 지질탐사학부에 입학했다.

그해 온성에서 중앙대학에 입학한 사람은 단 두 명이었다. 허 씨 외에 이과대학 입학생이 한 명 더 있었다. 온성군은 그런 동네였다.

● 돈에 성적을 뺏기던 시절
북한의 다양한 산업현장에서 활약하는 인재들을 키우는 김책공대는 학제가 길어 7년을 다녀야 졸업증을 받을 수 있다. 그런데 대학을 다니며 각종 공사현장에 끌려 다니다 보니 진도가 밀려 7년 안에 졸업하기가 어렵다. 허 씨도 학제보다 1년을 더 다녀 2000년에야 졸업할 수 있었다.

그가 대학을 다니던 1990년대 중반은 북한에서 고난의 행군 시기라 사방에서 아사자가 속출할 때였다. 대학 기숙사에서 주는 밥을 먹으면 굶어죽을 수밖에 없었다.

탄광마을에서 태어나 김책공대에 입학한 허 씨와, 어촌마을에서 태어나 김일성대에 입학한 기자는 평양에서 대학을 다닌 시기가 정확히 겹친다. 그래서 인터뷰 내내 떠올리기 싫은 추억들이 소환됐다. 몇 개를 소개하면 이런 식이다.

“1993년 공화국 창건 행사 때 김일성대 학생들은 깃발을 들고 김일성광장을 통과했습니다. 그 연습만 3개월 하면서 너무 힘들어 죽을 뻔 했죠.”

“김책공대는 촛불을 들고 김일성대를 따라갔죠. 저도 죽을 뻔 했어요.”

“제대군인인 학급 소대장, 청년단체비서 이런 사람들은 가난한데 공부 잘하는 학생들을 곁에 한둘씩 끼고 있죠. 그리곤 시험 때마다 자기 것도 좀 써달라고 사정하죠. 그걸 거절하기 어려워 저는 시험 칠 때마다 시험지 3개를 써주었어요. 다양한 볼펜을 준비해서 한 장은 정자로 쓰고, 다른 장은 흘겨 쓰고, 이런 식으로 답을 적어 교수가 뒤돌아서 있을 때 공부 못하는 제대군인들에게 몰래 건네주었죠. 대신 그들은 각종 비용을 걷을 때 나를 빼주기도 했고, 가끔 도시락도 두 개를 갖고 와서 허기진 배도 채워주었습니다.”

“저도 그랬어요. 국가졸업시험 때까지 남의 시험지를 작성해주었어요. 대신 저는 돈을 받았습니다. 방학 때 집에 가지도 않았지요. 온성까지 기차로 일주일 넘게 걸리는데 왔다갔다 시간 낭비가 크고, 또 가봐야 집에서 보태줄 수도 없으니 방학 때는 제대군인들 과외를 해주고 돈을 받았습니다.”

“저는 졸업장을 받고 나니, 3점짜리 과목이 몇 개 있었어요. 저는 대학 내내 3점을 받은 적이 없거든요. 5점 만점에 3점은 낙제를 겨우 면한 수준인데, 대학 교무부에 찾아가 싸우지도 않았어요. 어차피 이 체제가 싫어서 탈북하려 결심한 마당에 3점이 대수냐고 생각했죠.”

“저도 졸업증에 받지 않은 3점들이 있었어요. 권력자 부모를 둔 학생들은 좋은 곳에 가기 위해 대학 교무부 직원들에게 뇌물을 주고 컴퓨터에서 점수를 바꾸었어요. 5점 최우등생과, 4점 우등생, 3점 보통생의 전체 비율을 바꿀 수 없으니 5점으로 조작하려면 누군가의 점수를 빼앗아야 했죠. 제일 힘이 없는 우리가 점수를 빼앗긴 거였죠.”

최우등을 하고도, 돈과 권력이 없으면 3점 졸업증을 받아야 했던 시대. 허 씨와 기자는 그 시대에 평양에서 대학을 다녔다. 기숙사에서 밥을 몇 숟가락만 주던 때라 대학 시절을 떠올리면 배고팠던 기억밖에 없다. 졸업증을 받은 것만으로도 기적이었다. 뇌물로 성적을 조작하고, 남의 졸업논문을 베껴 써서 제출하는 상황은 지금도 별반 나아지진 않았다.


2018년 뉴질랜드로 한 달 동안 어학연수를 떠났던 시기의 모습.


● 북한의 측량기사들
아버지가 없는 가난한 탄광마을 출신의 김책공대 졸업생에게 좋은 직업이 기다릴 리 만무했다. 북한은 직업 선택의 자유가 없다. 대학을 졸업하면 중앙에서 어디로 가라고 임명한다.
권력과 부를 가진 집안에서 태어나면 대학 때 실컷 놀고도 중앙당이나 외화벌이 업체, 보위부 등 권력기관에 발령을 받는다. 가난한 자들의 운명은 그와 반대이다.

허 씨는 2000년 3월 대학을 졸업하면서 졸업증과 함께 측량기사 자격을 부여받았다. 그리고 평양 인근 대동군 시정노동자구에 있는 중앙측량단으로 발령을 받았다. 모두가 기피하는 직업이었다.

측량기사는 20㎏이 넘는 장비를 메고 매일 같이 산을 오르내려야 했다. 1000분의 1 오차 범위 내에서 지도에 점 하나를 찍는데 사나흘이 걸렸고, 한 구역을 측량하는데 2~3개월이 걸렸다. 깊은 산속에 천막을 치고 야인생활을 하기 일쑤였다. 그나마 현장기사는 배급을 받을 수 있어 다행이었다.

북한은 측량을 하는 기관이 두 개가 있다. 하나는 중앙측량단이고, 다른 하나는 인민무력부(군) 측지국이다. 그런데 진짜 좌표는 측지국이 갖고 있다. 중앙측량단의 좌표는 일부러 정확한 좌표와 다르게 작성한다. 좌표가 고급 비밀이기 때문에 외부에 정보가 새나가면 안된다는 이유 때문이다.

허 씨는 한국에 온 뒤 큰 허탈감을 느꼈다. 위성으로 GPS를 찍는 시대를 보니 그 무거운 장비를 메고 다니며 점 하나를 찍겠다고 며칠씩 바친 과거가 너무 허망하게 생각됐기 때문이다. 북한도 2005년경부터 러시아 위성항법체계인 ‘글로나스’의 도움을 받아 위성 측량을 시도했다고는 알려졌지만, 어느 정도 활용되는지는 아직 파악되지 않고 있다.

● 원시적 석탄채굴
허 씨는 2001년 말에 온성으로 돌아왔다. 뇌물이 없으면 이직도 힘든 세상이지만, 아버지도 없는데 어머니마저 아파서 쓰러졌다고 하니 집으로 보내준 것이다. 실제로 어머니를 간호할 사람은 허 씨 밖에 없었다.

새로 발령받은 직장은 풍인탄광 기술과였다. 그러나 할 일은 없었다. 1990년대 중반 고난의 행군 시기 온성 탄광들은 모두 침수됐다. 전기가 없어 물을 빼낼 수가 없었기 때문이다. 그렇게 몇 년을 방치하다보니 갱을 다시 사용할 수가 없었다.

탄광에 다니던 사람들은 먹고 살기 위해 다른 방법을 찾았다. 일제 때 했던 방식대로 얇은 탄층 채굴을 시작한 것이다. 삽과 곡괭이로 수직굴을 파들어 가는데, 운이 좋으면 8m에서 탄층을 만나기도 하지만, 운이 나쁘면 25m까지 파들어 간다. 탄층을 만나면, 그걸 따라 이번엔 가로로 파 들어간다. 탄층의 두께는 보통 0.8~1.2m였다. 양동이를 매단 도르래를 타고 수직굴에 들어가 몸을 돌리기도 어려운 좁은 공간에서 석탄을 파서 다시 양동이로 끌어올렸다. 도무지 현대인이라고 볼 수 없는 원시적 채탄방식이었지만, 그렇게 해서라도 석탄을 캐서 팔아야 장마당에서 먹을 것을 사올 수 있었다. 이런 수직굴을 온성에선 ‘노두’라고 불렀다. 온성에는 이런 노두가 수없이 많았다. 노두당 가족, 친척, 친구 등 5~7명이 팀을 이뤄 작업했는데, 이런 사람들을 ‘노두공’이라 불렀다. 북쪽은 날씨가 춥기 때문에 먹는 것 못지않게 석탄도 귀하다. 캐내면 파는 것은 큰 문제가 안됐다.

노두도 1년 내내 할 수 있는 것이 아니었다. 땅이 어는 1~3월에만 할 수 있었고, 봄이 와서 땅이 녹으면 수직갱이 버틸 수 없기 때문에 버려야 했다. 구멍을 방치해서도 안됐다. 단속기관 사람들이 찾아와 갱을 다시 메웠는지 조사한다. 단속할 권한이 있다는 것은 뇌물을 받을 수 있다는 것을 의미하기 때문에 얼렁뚱땅 넘어가지 않는다.

온성 전체에 이런 노두가 수없이 생겨났다 사라졌다. 겨울이면 할 일이 없는 농민들도 이 일에 매달렸다. 안전은 뒷전이라 붕괴와 추락, 가스질식 등으로 죽는 사람들이 계속 생겨났지만 큰 문제가 되진 않았다. 그 일을 하지 않으면 어차피 굶어죽기 때문이다.

허 씨가 소속된 기술과는 늘 각종 공사에 동원됐다. 도로 공사, 발전소 공사 등등 인력을 차출하는 공사는 끊이질 않았다. 김책공대에서 배운 지식을 쓸 곳은 없었다.

이렇게 살다가 어느새 결혼할 나이가 됐다. 그래도 허 씨는 명색이 탄광마을이 배출한 수재이고, 평양에서 김책공대까지 나왔던 터라 여성들에게 나름 인기가 있었다.

32살 때인 2006년 그는 10살 어린 여성과 결혼했다. 아내는 탄광 선전대 가수로 나름 인기가 좋았다. 이성적인 지식인과 감성적인 예술인의 조합이었다.

그렇지만 매력과 결혼생활은 별개였다. 결혼한 직후부터 둘은 다투는 일이 많았다. 서로가 서로를 견디기 어려워했다. 4년간의 결혼생활 끝에 둘은 이혼하기로 합의했다. 어린 딸은 아내가 키우기로 했다. 이때부터 그는 중국으로 눈을 돌렸다.


2022년 인천지하철 공사 현장에서 일하고 있는 허 씨.


● 두만강을 넘나들다
강 건너 연변 왕청에는 아버지의 삼촌과 사촌 등 친척들이 살고 있었다. 국경 근처라 많은 사람들이 중국을 드나들 때도 허 씨는 명색이 김책공대 졸업생인데 조국을 배반하면 안 된다는 마음이 컸다. 하지만 이혼 후 세상이 달리 보이기 시작했다. 이혼까지 한 마당에 눈치 볼 일도, 무서울 일도 없었다.

2010년 겨울 그는 국경경비대에 돈을 쥐어주고 두만강을 넘었다. 탈북한 것은 아니고, 친척에게 도움만 받자는 목적이었다. 저녁에 넘어가서 친척에게 전화를 하고 새벽이 되기 전에 다시 북으로 넘어왔다.

2012년 2월 그는 두 번째로 중국으로 갔다. 당시는 김정일이 사망한지 얼마 안됐던 때라 경계가 매우 심했다. 그는 북한군 군복을 입고 길을 떠났다. 군인은 초소에서 잘 단속하지 않았기 때문이다. 강을 따라 난 도로로 한참을 가다가 어둠이 내릴 때 바로 두만강을 넘었다. 중국 부락의 아무 집이나 들어가 왕청 친척에게 전화를 좀 하고 싶다고 했다. 전화를 하고 몇 시간 정도 머물렀는데 갑자기 공안이 들이닥쳤다. 집주인이 그를 도와주는 척하고는 신고를 했던 것이다. 알고 보니 중국은 그즈음부터 탈북자를 신고하면 포상금을 주는 제도를 운영하고 있었다. 특히 북한군 국경경비대가 수시로 건너와 사람까지 죽이며 노략질을 했기 때문에, 중국에서 북한군은 최고의 기피대상이었다.

탈북자들이 북송 전 대기하는 도문변방수용소에 끌려갔는데 며칠 뒤 보위부에서 차를 갖고 건너와 그를 싣고 갔다.

그렇지만 그는 예상과는 달리 20일 정도 가수감됐다가 석방됐다. 서류를 보니 중국에 친척이 다 있는 것도 확실하니, 도움 좀 받으려 넘어갔을 뿐이라는 그의 말이 인정됐다. 탈북은 배반이지만, 그의 경우엔 일탈 정도로 간주됐다.

보위부라고 해봐야 어차피 한동네 사는 아는 사람들이었는데, 그들은 지역이 배출한 인재의 경력을 망가뜨리고 싶지 않았는지 그다지 혹독하게 대하진 않았다.

하지만 허 씨 입장에선 아무 것도 이루지 못하고 잡혀오니 화가 났다. 보위부 감방에서 그는 강 건너편에서 일하다 잡혀왔다는 사람을 알게 됐다. 그는 자기가 일했던 중국 훈춘의 한 슈퍼의 이름을 알려주면서, 다시 건너가 자기 이름을 대면 사장이 고용해 줄 것이라고 했다.

감방에서 나온 허 씨는 다시 두만강을 넘었다. 세 번째 탈북이었다. 그는 감옥 동기가 알려준 슈퍼로 갔다. 왕청에 가지 않은 이유는 친척들은 별로 도와줄 의향이 없어 보였기 때문이다.
슈퍼는 중국과 나진선봉을 연결하는 도로 옆에 있었는데, 두만강 건너 허 씨네 동네가 약 10㎞ 밖에 빤히 바로 보였다. 허 씨가 내륙 깊이 들어가지 않은 이유는 딸을 데려오고 싶어서였다. 고향 가까이 있어야 집과 연락이 수월하다고 판단했다. 허 씨가 일하는 슈퍼에는 북한을 오가는 운전기사들이 수시로 드나들었다. 이들을 통해 그는 전처에게 휴대전화를 전달할 수 있었다.

● 자수해 감옥에 가다
그렇게 1년쯤 지났는데 사고가 생겼다. 중국 휴대전화를 받은 전처가 그걸 이용해 돈 벌려고 브로커 일을 하다가 보위부에 체포된 것이다. 전 남편마저 실종되니 보위부는 그녀에게 한국행을 기도했다는 누명을 씌웠다.

그 소식이 허 씨에게도 전달됐다. 그래도 4년 간 살았던 정도 있고, 어린 딸까지 있으니 모르는 척 할 수가 없었다. 그는 다시 북에 나가기 위해 자수하기로 결심했다. 자신이 나타나야 전 남편이 한국에 갔다는 누명을 벗길 수 있을 것 같았다.

슈퍼 주인은 전과자 출신이었는데, 그가 북으로 가겠다고 하자 감옥에서 나오는 자기의 비법을 전수해주었다. 폐 주변에 황산철을 주사하면 고열이 나고, 염증이 생긴다면서, 자신은 그 방법으로 병원에 호송됐다가 도망쳤다고 했다. 단 이 방법의 단점은 한 달 안에 황산철을 세척하지 못하면 생명을 잃을 수도 있다고 했다.

허 씨는 그의 조언에 따라 폐에 황산철을 네 군데나 주입했다. 그리고 스스로 공안에 자수한 뒤 2014년 4월 13일 북송됐다. 그런데 이번은 보위부도 호락호락하지 않았다. 불행하게도 그가 자수한 뒤 공안이 그의 숙소를 뒤져 소지품을 북송할 때 함께 보냈던 것이다. 그 속에는 한국 영사관, 한국인 전도사 등의 전화번호가 적힌 수첩이 있었다. 이게 화근이 됐다. 고문이 시작됐다.

중국 사장이 알려준 방법은 확실히 효과가 있었다. 폐가 곪아가기 시작했던 것이다. 염증으로 심한 열까지 나 거동을 못할 정도가 됐다. 보위부는 감방에서 죽이긴 싫었는지 그를 가석방으로 집에 보냈다.

그때가 5월 말이었다. 중국 사장이 당부한 폐를 씻어야 하는 한 달이 지난 것이다. 집에 가서 이러저런 치료를 해봤지만 점점 악화됐다. 이렇게 지내다간 죽을 것 같았다. 그는 황산철 주사를 맞은 지 네 달이 지난 8월 23일 다시 중국으로 넘어왔다. 이번엔 하얼빈에 들어가 식당에 취직해 치료에만 전념했다. 하지만 몸은 점점 더 쇠약해졌다.

몸을 운신하기 어려워지자 허 씨는 죽더라도 고향에 가서 죽겠다고 생각했다. 그해 11월말 그는 다시 연길에 왔다. 연길에서 혹시나 도움을 받지 않을까 싶어 처음으로 교회에 찾아갔는데, 거기서 집도 제공하고 치료도 해주었다. 12월 말 허 씨는 산소 호흡기에 의지하는 신세가 됐다. 쇼크도 수시로 왔다. 교회에선 한국에 가서 치료를 받아야 살 수 있다며 한국행선을 주선해주었다.


교회 목회자가 될 꿈을 꾸던 당시의 허 씨. 2017년 뉴욕에서 찍은 사진이다.


● 최초로 남북 측량기사 자격 모두 획득
2015년 2월 그는 여러 탈북민들과 함께 한국으로 떠났다. 동남아 정글에서 일행을 따라갈 수 없을 때 친한 동생이 그를 업고 산을 넘었다. 우여곡절 끝에 한국에 도착했지만 다음날 적십자병원에 실려갔다. 이곳에서 4개월 동안 치료를 받다보니 하나원도 나오지 못했다.

허 씨는 인천에 거주지가 배정됐다. 건강이 너무 악화돼 일을 할 수 없는 상태라 2016년말까지 병원에 다니며 통원치료를 받았다. 하나원을 나온 다른 사람들이 하나둘 취직해 일자리를 얻는 것을 보고 조급한 마음에 노량진에서 공무원 시험을 준비하기도 했지만, 건강이 악화돼 다시 병원에 실려 가기도 했다. 한국의 의술은 역시 대단했다. 2년이라는 오랜 시간이 걸리긴 했지만, 끝내 허 씨를 완치시켰다.

치료 받는 기간에 허 씨는 목회자의 길을 걸어볼까 싶어 1년 남짓 성경공부도 했고, 화장품 회사에 취직해 일도 해봤지만 적성에 맞지 않았다.

많은 고민 끝에 북에서 배운 측량기사 일을 다시 해보려 알아보니, 남쪽도 측량기사는 일도 힘들고 보상도 적었다. 그렇지만 적성에 맞지 않는 곳에서 시간을 낭비하는 것보다는 그래도 자신있는 분야를 파보자는 생각에 2018년 한 엔지니어링 회사에 취직했다. 취직은 했어도 아무런 건설기술인 등급도 받지 못했기 때문에 회사에서 가장 많은 나이임에도 허드레 일만 해야 했다. 연봉은 2300만 원이었다. 그는 앞으로 살아가려면 자격증이 필수라는 것을 깨달았다. 어떤 자격을 갖고 있고, 어떤 경력을 쌓는가에 따라 연봉도 결정됐다.

허 씨는 회사에 다니며 1년 동안은 토목계측에 대한 용어부터 수첩에 적어 기본적인 지식을 배웠다. 그리고 이듬해 대구과학대에 입학했다. 일도 하면서 대학 공부까지 하려니 야간반을 다닐 수밖에 없었는데, 전국의 측량 관련 야간반 중에 대구가 그나마 가까웠다.

가깝다고 해도 당시 일을 하던 포항의 건설현장에서 150㎞나 떨어져 있었다. 일주일에 서너번씩 왕복 300㎞를 달려 수업을 듣고 오는 일은 결코 쉽지 않았다.

돌아오는 길에 너무 피곤해 졸음 쉼터에서 쪽잠을 자기 일쑤였다. 잠깐 눈을 붙인다는 것이 해가 중천에 걸릴 때까지 잔적도 있다.

허 씨는 대학 공부와 병행해 자격증 시험도 준비했다. 3년 동안 주경야독의 삶을 끈질기게 이어나간 끝에 그는 2022년 대학을 우수한 성적으로 졸업하고, 동시에 측량 및 지형공간정보기사와 토목기사 자격증을 받을 수 있었다. 허 씨는 남과 북에서 측량기사 자격을 각각 받은 최초의 사례다.

이러한 자격증과 경력을 인정받아 현재 허 씨는 건설기술인협회에서 고급기술인으로 인정받고 있으며, 300억 원 규모 이상의 토목공사를 책임지고 할 수 있다.


2019년 여수에서 전력 케이블용 해저터널 공사장에서 작업하는 모습. 뒷모습이 보이는 사람이 허 씨이다.


● “배워야 살 수 있다”
허 씨는 김책공대에서 무려 8년이나 공부를 했지만 여기선 모든 것을 다시 배워야 했다고 말했다.

“남쪽에 오니 용어는 물론, 장비나 자재 등 모든 것이 북한과 달랐습니다. 비유하면 북에서 통나무로 집을 짓는 법을 배웠는데, 남쪽에 오니 시멘트로 아파트를 짓는 격이죠. 그럼 집 짓는 법을 처음부터 다시 배워야하죠. 물론 북한 김책공대 과정이 전혀 의미 없는 것은 아닙니다. 북한에선 가장 기본적인 것, 즉 공부하는 방법을 배운 것 같습니다.”

그는 남북은 통합성에서도 차이가 크다고 했다.

“여기는 한 가지를 하려면 열 가지를 알아야 합니다. 많은 것들이 서로 연결돼 있어 전체적으로 진행되죠. 반면 북한은 직무가 매우 세분화돼 있고, 맡겨진 것만 하면 됐어요.”

자격증과 경력을 쌓고 나니 연봉도 빠르게 올라갔다. 김책공대까지 입학했던 북한 시골 수재가 다시 자신의 두뇌와 재능을 발휘하는 데는 오랜 시간이 걸리지 않았다. 2300만 원에서 시작했던 연봉은 6년 만에 3배 이상 높아졌다.

그는 여기에 만족하지 않았다. 그는 올해 3월 다시 대구대 공간정보전문기술 석사과정에 입학했다. 토질 및 기초기술사 자격을 취득하기 위해서다. 다른 자격증과는 달리 이 자격은 받기가 매우 어렵다. 허 씨의 계획은 향후 5년 안에 석사와 함께 기술사 자격을 획득하는 것이다.

그렇다고 공부만 할 수는 없는 일. 요즘 허 씨는 마산만에서 스팀배관 부설을 위한 해저터널을 뚫는 작업을 하고 있다. 그의 직책은 공사차장. 터널 시공 품질을 총괄하는 중요한 자리다.

건설 현장은 매우 거칠고 다툼도 많다. 외국인 근로자들도 많아 의사소통의 문제도 많다. 많은 어려움이 있지만 허 씨는 포기하지 않고 한 걸음씩 나가고 있다.

“거친 현장이라서 그런지 북에서 왔다고 우습게 보는 사람들이 많아요. 아직도 저는 ‘나는 여전히 이방인이구나’라는 생각을 하고 있습니다. 그렇지만 실력은 남에게 뒤진다고 생각하지 않습니다. 여러 곳에서 이직 제안도 많이 받습니다.”

2023년 남북하나재단이 주관한 정착사례 발표대회에서 그는 최우수상인 통일부 장관상을 받았다. 하지만 허 씨는 지금은 정착의 첫 발자국을 뗀 것에 불과하다고 했다.

그는 자신의 이름으로 회사를 만들어 키우는 것을 다음 목표로 정했다. 그 목표가 달성되면 또 할 일이 있다. 나아가 통일이 되면 할 계획도 미리 생각해두었다.

“저는 북에 돌아가 여기서 배운 기술을 북에 전수할 겁니다. 한국은 토목 공사를 할 때 측량, 시공, 품질까지 다 알아야 합니다. 통일 되면 교통인프라를 다시 정리해야 할 것인데, 북한에서 대학을 다녔던 동창들은 통합성에 있어 크게 떨어집니다. 이런 것을 가르쳐야죠. 그리고 남북에서 모두 측량기사 자격을 받은 유일한 사람이니 다른 꿈도 있습니다. 남북공간정보 통합지도체계를 만드는 것도 중요한 목표입니다.”

꿈을 말할 때 그는 가장 행복한 표정이었다.'''

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma


# 텍스트를 600자 단위로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

texts = text_splitter.create_documents([script])

In [ ]:
len(texts)

22

In [ ]:
texts[0]

Document(metadata={}, page_content='허수현은 한반도 최북단 탄광마을이 낳은 수재였다. 그는 고등학교 졸업 직전 북한 전체에서 700명에게만 수여하는 ‘7.15 최우등상’을 수상했다. 7.15최우등상은 김정일이 평양 남산고급중학교를 졸업한 날인 1960년 7월 15일을 기념해 1987년에 만들어진 상이다.\n\n지금은 이 상이 특권층 자식들을 대학에 보내기 위한 발판이 돼 각종 비리와 뇌물로 얼룩져 있지만, 상이 제정된 초기 몇 년은 정말 공부 잘하는 사람에게만 수여됐다. 상을 받게 되면 곧바로 중앙급 대학에 진학할 수 있었다. 북한에선 고등중학교 졸업생 중 20% 미만이 대학이나 전문학교에 갈 수 있다는 것을 감안하면 엄청난 특혜였다.\n\n허 씨도 김책공업종합대학(김책공대)에 입학해 8년이나 공부했다. 그리고 그때 배운 지식을 활용해 지금은 한반도 최남단인 경남 마산의 해저터널 공사장에서 시공품질을 관리하는 공사차장으로 일하고 있다. 김책공대 졸업생이 네 번의 탈북을 반복한 뒤 건강이 악화돼 남의 등에 업혀 동남아 정글을 넘어 한국까지 오게 되고, 이후 남과 북에서 동시에 측량기사 자격을 받은 최초의 기술자가 돼 해저터널 공사장에서 일하게 되기까지 삶의 과정은 결코 순탄하지 않았다.')

In [ ]:
# Chroma 를 통해 벡터 저장소 생성
chroma_db = Chroma.from_documents(texts, OpenAIEmbeddings())

In [ ]:
# 유사도 검색(쿼리)
similar_docs = chroma_db.similarity_search("주인공은 어디서 태어났는가?")
print(similar_docs[0].page_content)

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.


## 5-3. VectorStoreRetriever: as_retreiver()

`VectorStoreRetreiver`는 벡터 저장소를 사용하여 문서를 검색하는 Retriever 입니다.

In [ ]:
# retriever 생성
retriever = chroma_db.as_retriever()

# similarity_search 를 통해 유사도 높은 1개 문서를 검색
relevant_docs = retriever.get_relevant_documents("주인공은 어디서 태어났는가?")

print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
print(relevant_docs[0].page_content)

<ipython-input-100-273597ce08cc>:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents("주인공은 어디서 태어났는가?")


문서의 개수: 4
[검색 결과]

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.


In [ ]:
# retriever 생성
retriever = chroma_db.as_retriever(search_kwargs={"k": 1})

# similarity_search 를 통해 유사도 높은 1개 문서를 검색
relevant_docs = retriever.get_relevant_documents("주인공은 어디서 태어났는가?")

print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
print(relevant_docs[0].page_content)

문서의 개수: 1
[검색 결과]

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.


**search_type**  
search_type 매개변수에 검색 알고리즘을 지정할 수 있습니다.

- similarity(기본값), mmr, similarity_score_threshold 등의 옵션을 지정할 수 있습니다.
- 아래에서 지정 옵션 별 차이에 대해 다룹니다.
search_type='similarity'

기본 값으로 설정되어 있습니다. vector store 의 유사도 알고리즘 기반으로 상위 K개의 문서를 검색합니다.

In [ ]:
retriever = chroma_db.as_retriever(search_type="similarity", search_kwargs={"k": 2})
relevant_docs = retriever.get_relevant_documents("주인공은 어디서 태어났는가?")
print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
for i in range(len(relevant_docs)):
    print(relevant_docs[i].page_content)
    print("===" * 20)

문서의 개수: 2
[검색 결과]

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.
32살 때인 2006년 그는 10살 어린 여성과 결혼했다. 아내는 탄광 선전대 가수로 나름 인기가 좋았다. 이성적인 지식인과 감성적인 예술인의 조합이었다.

그렇지만 매력과 결혼생활은 별개였다. 결혼한 직후부터 둘은 다투는 일이 많았다. 서로가 서로를 견디기 어려워했다. 4년간의 결혼생활 끝에 둘은 이혼하기로 합의했다. 어린 딸은 아내가 키우기로 했다. 이때부터 그는 중국으로 눈을 돌렸다.


2022년 인천지하철 공사 현장에서 일하고 있는 허 씨.


● 두만강을 넘나들다
강 건너 연변 왕청에는 아버지의 삼촌과 사촌 등 친척들이 살고 있었다. 국경 근처라 많은 사람들이 중국을 드나들 때도 허 씨는 명색이 김책공대 졸업생인데 조국을 배반하면 안 된다는 마음이 컸다. 하지만 이혼 후 세상이 달리 보이기 시작했다. 이혼까지 한 마당에 눈치 볼 일도, 무서울 일도 없었다.

2010년 겨울 그는 국경경비대에 돈을 쥐어주고 두만강을 넘었다. 탈북한 것은 아니고, 친척에게 도움만 받자는 

**Maximal Marginal Relevance(MMR) 검색**

MMR은 쿼리와 관련된 항목을 검색하면서 동시에 내용의 중복을 최소화하는 기법입니다.

이 방법은 단순히 관련성이 높은 항목들을 선택하는 대신, 관련성과 다양성 사이의 균형을 찾는 데 중점을 둡니다.

**MMR 에 대해 이해해보기**

모임에 참석해 친구를 위해 새로운 사람들을 소개하는 상황을 상상해 보세요. 친구가 만나고 싶어하는 사람들의 특징(embedding vector)을 알고 있습니다.

MMR 방식을 적용하면 다음과 같은 접근법을 사용합니다:

1. 모임에 있는 모든 사람들의 프로필(list of embedding vector)을 살펴봅니다.
2. 친구의 관심사와 특성에 부합하는 사람을 찾아 소개합니다.
3. 그 후, 다시 참여한 사람들을 살펴보되, 이번에는 이미 소개한 사람과는 다른 특성을 가진 사람을 찾습니다.
4. 여기서 lambda mult 매개변수는 친구의 취향과 새로운 특성 사이의 균형을 조정하는 역할을 합니다.
5. 이 과정을 친구가 만나길 원하는 사람들의 수(k 매개변수)에 도달할 때까지 반복합니다.
6. 마지막으로, 친구가 만난 사람들의 목록을 제공합니다.  

이 예시를 통해 MMR이 **관련성 높은 항목을 선택** 하는 동시에 **내용의 다양성을 유지**하려는 방식을 이해할 수 있습니다.

**search_type='mmr'**을 통해 MMR 을 사용한 검색을 수행할 수 있습니다.

In [ ]:
retriever = chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 2})
relevant_docs = retriever.get_relevant_documents("주인공은 어디서 태어났는가?")
print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
for i in range(len(relevant_docs)):
    print(relevant_docs[i].page_content)
    print("===" * 20)

문서의 개수: 2
[검색 결과]

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.
“저는 졸업장을 받고 나니, 3점짜리 과목이 몇 개 있었어요. 저는 대학 내내 3점을 받은 적이 없거든요. 5점 만점에 3점은 낙제를 겨우 면한 수준인데, 대학 교무부에 찾아가 싸우지도 않았어요. 어차피 이 체제가 싫어서 탈북하려 결심한 마당에 3점이 대수냐고 생각했죠.”

“저도 졸업증에 받지 않은 3점들이 있었어요. 권력자 부모를 둔 학생들은 좋은 곳에 가기 위해 대학 교무부 직원들에게 뇌물을 주고 컴퓨터에서 점수를 바꾸었어요. 5점 최우등생과, 4점 우등생, 3점 보통생의 전체 비율을 바꿀 수 없으니 5점으로 조작하려면 누군가의 점수를 빼앗아야 했죠. 제일 힘이 없는 우리가 점수를 빼앗긴 거였죠.”

최우등을 하고도, 돈과 권력이 없으면 3점 졸업증을 받아야 했던 시대. 허 씨와 기자는 그 시대에 평양에서 대학을 다녔다. 기숙사에서 밥을 몇 숟가락만 주던 때라 대학 시절을 떠올리면 배고팠던 기억밖에 없다. 졸업증을 받은 것만으로도 기적이었다. 뇌물로 성적을 조작하고, 남의 

search_type='similarity_score_threshold' 을 지정하면 score_threshold 기준을 충족하는 유사도 문서가 반환됩니다.

만약 {"k": 3} 이지만, {"score_threshold": 0.5} 로 설정되었는데 score 가 0.5를 넘는 문서가 2개 밖에 없다면, 결과는 3개 문서가 아닌 2개 문서가 반환됩니다.

In [ ]:
retriever = chroma_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 2, "score_threshold": 0.5},
)
relevant_docs = retriever.get_relevant_documents("주인공은 어디서 태어났는가?")
print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
for i in range(len(relevant_docs)):
    print(relevant_docs[i].page_content)
    print("===" * 20)

문서의 개수: 2
[검색 결과]

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.
32살 때인 2006년 그는 10살 어린 여성과 결혼했다. 아내는 탄광 선전대 가수로 나름 인기가 좋았다. 이성적인 지식인과 감성적인 예술인의 조합이었다.

그렇지만 매력과 결혼생활은 별개였다. 결혼한 직후부터 둘은 다투는 일이 많았다. 서로가 서로를 견디기 어려워했다. 4년간의 결혼생활 끝에 둘은 이혼하기로 합의했다. 어린 딸은 아내가 키우기로 했다. 이때부터 그는 중국으로 눈을 돌렸다.


2022년 인천지하철 공사 현장에서 일하고 있는 허 씨.


● 두만강을 넘나들다
강 건너 연변 왕청에는 아버지의 삼촌과 사촌 등 친척들이 살고 있었다. 국경 근처라 많은 사람들이 중국을 드나들 때도 허 씨는 명색이 김책공대 졸업생인데 조국을 배반하면 안 된다는 마음이 컸다. 하지만 이혼 후 세상이 달리 보이기 시작했다. 이혼까지 한 마당에 눈치 볼 일도, 무서울 일도 없었다.

2010년 겨울 그는 국경경비대에 돈을 쥐어주고 두만강을 넘었다. 탈북한 것은 아니고, 친척에게 도움만 받자는 

**search_kwargs**
search_kwargs에 다양한 옵션을 지정할 수 있습니다.

다음의 search_kwargs 로 Retriever 의 검색 결과를 세부조정할 수 있습니다.

* k: "k" 매개변수로 찾을 문서의 개수를 지정할 수 있습니다. 만약, {"k": 3} 지정한다면 3개의 유사도 높은 문서만 선택하겠다는 의미 입니다. 기본 값은 1입니다.
* score_threshold: Minimum relevance threshold for similarity_score_threshold
* fetch_k: Amount of documents to pass to MMR algorithm (Default: 20)
* lambda_mult: Diversity of results returned by MMR. 1 for minimum diversity and 0 for maximum. (Default: 0.5)
* filter: document를 메타데이터 기준으로 필터링 합니다. (예시) search_kwargs={'filter': {'paper_title':'GPT-4 Technical Report'}

In [ ]:
retriever = chroma_db.as_retriever(
    search_type="mmr", search_kwargs={"k": 2, "fetch_k": 10, "lambda_mult": 0.75}
)
relevant_docs = retriever.get_relevant_documents("주인공은 어디서 태어났는가?")
print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
for i in range(len(relevant_docs)):
    print(relevant_docs[i].page_content)
    print("===" * 20)

문서의 개수: 2
[검색 결과]

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.
32살 때인 2006년 그는 10살 어린 여성과 결혼했다. 아내는 탄광 선전대 가수로 나름 인기가 좋았다. 이성적인 지식인과 감성적인 예술인의 조합이었다.

그렇지만 매력과 결혼생활은 별개였다. 결혼한 직후부터 둘은 다투는 일이 많았다. 서로가 서로를 견디기 어려워했다. 4년간의 결혼생활 끝에 둘은 이혼하기로 합의했다. 어린 딸은 아내가 키우기로 했다. 이때부터 그는 중국으로 눈을 돌렸다.


2022년 인천지하철 공사 현장에서 일하고 있는 허 씨.


● 두만강을 넘나들다
강 건너 연변 왕청에는 아버지의 삼촌과 사촌 등 친척들이 살고 있었다. 국경 근처라 많은 사람들이 중국을 드나들 때도 허 씨는 명색이 김책공대 졸업생인데 조국을 배반하면 안 된다는 마음이 컸다. 하지만 이혼 후 세상이 달리 보이기 시작했다. 이혼까지 한 마당에 눈치 볼 일도, 무서울 일도 없었다.

2010년 겨울 그는 국경경비대에 돈을 쥐어주고 두만강을 넘었다. 탈북한 것은 아니고, 친척에게 도움만 받자는 

## 5-4. FAISS

In [ ]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00


FAISS는 semantic search를 도와주는 Meta(구 Facebook)에서 만든 라이브러리 입니다.

* 링크: https://github.com/facebookresearch/faiss
리이선스는 **MIT License** 입니다.

**[참고]**

MIT 라이선스는 자유롭고 유연한 소프트웨어 라이선스 중 하나입니다. MIT 라이선스 하에 배포된 소프트웨어는 **상업적으로 이용**할 수 있습니다

주요 특징은 다음과 같습니다:

* 간결함과 넓은 범위의 사용 허가: MIT 라이선스는 매우 간결하며, 사용자에게 소프트웨어를 거의 제한 없이 사용, 복사, 수정, 합병, 출판, 배포, 하위 라이선스 부여, 판매할 수 있는 권리를 부여합니다.

* 저작권과 라이선스 고지 유지 요구: 라이선스는 사용자가 MIT 라이선스하에 배포된 모든 복사본과 상당한 부분에 원래의 저작권 고지와 이 라이선스 고지를 포함하도록 요구합니다.

* 책임의 부인: MIT 라이선스는 소프트웨어가 '있는 그대로' 제공되며, 소프트웨어의 사용으로 인한 어떠한 보증도 제공하지 않습니다. 이는 소프트웨어 사용으로 인한 모든 위험은 사용자가 부담한다는 것을 의미합니다.

MIT 라이선스는 그 간결함과 유연성으로 인해 많은 오픈 소스 프로젝트와 상업적 프로젝트에서 널리 사용됩니다. 이 라이선스는 소프트웨어 개발자가 자신의 소프트웨어를 거의 제한 없이 공유할 수 있도록 해주며, 소프트웨어 산업 전반에 걸쳐 협력과 혁신을 촉진하는 데 기여하고 있습니다.

API 사용방법은 Chroma와 동일합니다. (동일한 인터페이스를 제공합니다.)

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS


# 텍스트를 600자 단위로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

texts = text_splitter.create_documents([script])

faiss_db = FAISS.from_documents(texts, OpenAIEmbeddings())

In [ ]:
# 유사도 검색(쿼리)
similar_docs = faiss_db.similarity_search("주인공은 어디서 태어났는가?")

print(f"문서의 개수: {len(similar_docs)}")
print("[검색 결과]\n")
print(similar_docs[0].page_content)

문서의 개수: 4
[검색 결과]

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.


In [ ]:
# 데이터베이스를 검색기로 사용하기 위해 retriever 변수에 할당합니다.
retriever = faiss_db.as_retriever()

In [ ]:
# 검색 질의를 사용하여 관련 문서를 검색합니다.
query = "주인공은 어디서 태어났는가?"
docs = retriever.invoke(query)

In [ ]:
# docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(docs[0].page_content)

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.


FAISS에는 몇 가지 특정 메서드가 있습니다.

그 중 하나는 similarity_search_with_score로, 문서뿐만 아니라 쿼리와 문서 간의 거리 점수도 반환할 수 있습니다.

반환되는 거리 점수는 L2 거리입니다. 따라서 점수가 낮을수록 더 좋은 결과 입니다.

db.similarity_search_with_score(query) 메서드를 사용하여 질의와 유사한 문서를 검색하고 유사도 점수와 함께 반환합니다.

반환된 결과는 (문서, 점수) 튜플의 리스트 형태로 구성됩니다

In [ ]:
# 쿼리와 유사한 문서를 검색하고 유사도 점수와 함께 반환합니다.
docs_and_scores = faiss_db.similarity_search_with_score(query)
content, score = docs_and_scores[0]  # 문서와 점수 리스트에서 첫 번째 요소를 선택합니다
print("[Content]")
print(content.page_content)  # 선택된 문서의 page_content 속성을 출력합니다
print("\n[Score]")
print(score)  # 선택된 문서의 점수를 출력합니다

[Content]
2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.

[Score]
0.33392143


similarity_search_by_vector 함수를 사용하면 주어진 임베딩 벡터와 유사한 문서를 검색할 수 있습니다.

이 함수는 문자열 대신 임베딩 벡터를 매개변수로 받아들입니다.

In [ ]:
# 질의를 임베딩 벡터로 변환합니다.
query = "주인공은 어디서 태어났는가?"
embedding_vector = embeddings.embed_query(query)
# 임베딩 벡터를 사용하여 유사도 검색을 수행하고, 문서와 점수를 반환합니다.
docs_and_scores = faiss_db.similarity_search_by_vector(embedding_vector)
docs_and_scores[0]  # 문서와 점수 리스트에서 첫 번째 요소를 선택합니다

Document(metadata={}, page_content='2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.\n\n\n● 신분 상승의 꿈\n그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.\n\n온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.\n\n탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.\n부친은 늘 허 씨에게 “너는 공부를 잘해 꼭 신분 상승을 해야 한다”고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.')

## 5-6. 앙상블 검색기

In [ ]:
!pip install rank_bm25

EnsembleRetriever는 여러 retriever를 입력으로 받아 get_relevant_documents() 메서드의 결과를 앙상블하고, Reciprocal Rank Fusion 알고리즘을 기반으로 결과를 재순위화합니다.

서로 다른 알고리즘의 장점을 활용함으로써, EnsembleRetriever는 단일 알고리즘보다 더 나은 성능을 달성할 수 있습니다.

가장 일반적인 패턴은 sparse retriever (예: BM25)와 dense retriever (예: embedding similarity)를 결합하는 것인데, 이는 두 retriever의 장점이 상호 보완적이기 때문입니다. 이를 "hybrid search" 라고도 합니다.

Sparse retriever는 키워드를 기반으로 관련 문서를 찾는 데 효과적이며, dense retriever는 의미적 유사성을 기반으로 관련 문서를 찾는 데 효과적입니다.

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

두 개의 문서 리스트(doc_list_1과 doc_list_2)를 정의합니다.  
EnsembleRetriever를 초기화하여 BM25Retriever와 FAISS 검색기를 결합합니다. 각 검색기의 가중치를 설정됩니다.

In [ ]:
# 비타민 별 섭취할 수 있는 음식 정보
doc_list_1 = [
    "비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.",
    "비타민B : 전곡물, 콩, 견과류, 육류 등 다양한 식품에서 찾을 수 있습니다.",
    "비타민C : 오렌지, 키위, 딸기, 브로콜리, 피망 등의 과일과 채소에 많이 들어 있습니다.",
    "비타민D : 연어, 참치, 버섯, 우유, 계란 노른자 등에 함유되어 있습니다.",
    "비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.",
]

# 비타민 별 효능 정보
doc_list_2 = [
    "비타민A : 시력과 피부 건강을 지원합니다.",
    "비타민B : 에너지 대사와 신경계 기능을 돕습니다.",
    "비타민C : 면역 체계를 강화하고 콜라겐 생성을 촉진합니다.",
    "비타민D : 뼈 건강과 면역 체계를 지원합니다.",
    "비타민E : 항산화 작용을 통해 세포를 보호합니다.",
]


# bm25 retriever와 faiss retriever를 초기화합니다.
bm25_retriever = BM25Retriever.from_texts(
    # doc_list_1의 텍스트와 메타데이터를 사용하여 BM25Retriever를 초기화합니다.
    doc_list_1,
    metadatas=[{"source": 1}] * len(doc_list_1),
)
bm25_retriever.k = 1  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

embedding = OpenAIEmbeddings()  # OpenAI 임베딩을 사용합니다.
faiss_vectorstore = FAISS.from_texts(
    # doc_list_2의 텍스트와 임베딩, 메타데이터를 사용하여 FAISS 벡터 저장소를 초기화합니다.
    doc_list_2,
    embedding,
    metadatas=[{"source": 2}] * len(doc_list_2),
)
# FAISS 벡터 저장소를 사용하여 retriever를 생성하고, 검색 결과 개수를 1로 설정합니다.
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 1})

# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    # BM25Retriever와 FAISS retriever를 사용하여 EnsembleRetriever를 초기화하고, 각 retriever의 가중치를 0.6:0.4로 설정합니다.
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.6, 0.4],
    search_type="mmr",
)

ensemble_retriever 객체의 get_relevant_documents() 메서드를 호출하여 관련성 높은 문서를 검색합니다.

In [ ]:
# 검색 결과 문서를 가져옵니다.
query = "비타민A 의 효능은?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(metadata={'source': 1}, page_content='비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.'), Document(metadata={'source': 2}, page_content='비타민A : 시력과 피부 건강을 지원합니다.')]

[BM25 Retriever]
 [Document(metadata={'source': 1}, page_content='비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.')]

[FAISS Retriever]
 [Document(metadata={'source': 2}, page_content='비타민A : 시력과 피부 건강을 지원합니다.')]



In [ ]:
# 검색 결과 문서를 가져옵니다.
query = "시력에 좋은 비타민은?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(metadata={'source': 1}, page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.'), Document(metadata={'source': 2}, page_content='비타민A : 시력과 피부 건강을 지원합니다.')]

[BM25 Retriever]
 [Document(metadata={'source': 1}, page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.')]

[FAISS Retriever]
 [Document(metadata={'source': 2}, page_content='비타민A : 시력과 피부 건강을 지원합니다.')]



In [ ]:
# 검색 결과 문서를 가져옵니다.
query = "비타민E 는 어떻게 섭취할 수 있나요?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(metadata={'source': 1}, page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.'), Document(metadata={'source': 2}, page_content='비타민E : 항산화 작용을 통해 세포를 보호합니다.')]

[BM25 Retriever]
 [Document(metadata={'source': 1}, page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.')]

[FAISS Retriever]
 [Document(metadata={'source': 2}, page_content='비타민E : 항산화 작용을 통해 세포를 보호합니다.')]

